# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

## 🔧 설정 정의하기

### 3-1. Special Token 동적 수집 및 등록
- 학습 데이터에서 실제 등장하는 모든 마스킹/발화자 토큰을 자동 수집
- `additional_special_tokens`로 등록하여 토큰이 분해되지 않도록 함
- `model.resize_token_embeddings(len(tokenizer))` 호출로 임베딩 사이즈 조정

### 3-2. 프롬프트 빌더 (발화자/turn 구조 활용)
- `build_summary_prompt()`: 요약 태스크를 명시적으로 알려주는 프롬프트 추가
- `prompt_style` 옵션: "default", "speaker_aware", "minimal", "none"
- 발화자 수에 따른 동적 프롬프트 생성 가능

### 4. 구어체 전처리 (슬랭/감탄사/의성어 정규화)
- `normalize_slang()`: ㅋㅋ→웃기다, ㅠㅠ→슬프다 등 구어체→문어체 변환
- 요약문이 문어체이므로, 모델이 직접 해석하는 것보다 미리 변환하는 게 효과적
- `SLANG_MAP`: 약 50개 이상의 슬랭/감탄사/인터넷 용어 매핑
- `normalize_repeated_chars()`: 과도한 반복 문자 정규화 (ㅋㅋㅋㅋㅋㅋ → ㅋㅋㅋㅋ)

In [2]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [ ]:
# ============================================================================
# 3-1. Special Token 동적 수집 및 등록
# ============================================================================
import re
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# 데이터 경로 정의 (커널 재시작 후에도 동작하도록)
data_path = "./data/"

# 마스킹 토큰 & 발화자 토큰 패턴 정의
MASK_PAT = r"#\w+#"      # 모든 #...# 형태의 토큰
PERSON_PAT = r"#Person\d+#"  # 발화자 토큰만
# 6-1, 6-2에서 활용할 개인정보 마스킹 패턴 (발화자 제외)
PII_MASK_PAT = r"#(?!Person\d+#)\w+#"  # #Person1# 등 제외한 마스킹 토큰

def get_mask_tokens(text):
    """텍스트에서 모든 마스킹 토큰 추출 (#Word# 형태)"""
    return re.findall(MASK_PAT, text)

def get_person_tokens(text):
    """텍스트에서 발화자 토큰만 추출 (#PersonN# 형태)"""
    return re.findall(PERSON_PAT, text)

def get_pii_mask_tokens(text):
    """텍스트에서 개인정보 마스킹 토큰만 추출 (발화자 제외)"""
    return re.findall(PII_MASK_PAT, text)

def collect_special_tokens_from_data(dialogues):
    """
    학습 데이터에서 실제 등장하는 모든 special token 수집
    - #Person1#, #Person2# 등 발화자 토큰
    - #PhoneNumber#, #Address# 등 마스킹 토큰
    """
    all_tokens = set()
    for d in dialogues:
        all_tokens.update(get_mask_tokens(d))
    return sorted(list(all_tokens))

# Train 데이터에서 special tokens 수집
train_file = os.path.join(data_path, 'train.csv')
train_df_for_tokens = pd.read_csv(train_file)
DYNAMIC_SPECIAL_TOKENS = collect_special_tokens_from_data(train_df_for_tokens['dialogue'])

print("=" * 60)
print("🔖 동적으로 수집된 Special Tokens:")
print("=" * 60)
print(f"총 {len(DYNAMIC_SPECIAL_TOKENS)}개: {DYNAMIC_SPECIAL_TOKENS}")

# ============================================================================
# 6. 정규표현식 기반 패턴 활용 전략
# ============================================================================
# 6-1. 화자 수를 명시적으로 프롬프트에 포함
# 6-2. 마스킹된 정보(#PhoneNumber# 등)를 그대로 유지하라고 지시
# ============================================================================

def analyze_dialogue_structure(dialogue_text):
    """
    대화 구조 분석: 화자 수, 마스킹 토큰 정보 추출
    
    Returns:
        dict: {
            'speakers': set of speaker tokens,
            'num_speakers': number of speakers,
            'pii_masks': set of PII masking tokens,
            'has_pii_masks': whether PII masks exist
        }
    """
    speakers = set(get_person_tokens(dialogue_text))
    pii_masks = set(get_pii_mask_tokens(dialogue_text))
    
    return {
        'speakers': speakers,
        'num_speakers': len(speakers) if speakers else 2,  # 기본값 2명
        'pii_masks': pii_masks,
        'has_pii_masks': len(pii_masks) > 0,
        'pii_mask_examples': list(pii_masks)[:3]  # 예시 최대 3개
    }

def build_speaker_aware_prompt(num_speakers):
    """
    6-1. 화자 수를 명시적으로 알려주는 프롬프트 생성
    - "한 사람 이야기만 요약하는" 버그 방지
    """
    if num_speakers == 1:
        return "다음은 1명의 독백입니다. "
    elif num_speakers == 2:
        return f"다음은 2명이 나눈 대화입니다. 두 사람의 입장을 균형 있게 반영해서 요약해 주세요. "
    else:
        return f"다음은 {num_speakers}명이 나눈 대화입니다. 각 사람의 입장을 균형 있게 반영해서 요약해 주세요. "

def build_pii_mask_instruction(pii_masks, show_examples=True):
    """
    6-2. 마스킹된 정보 유지 지시 프롬프트 생성
    - 요약문에서 엉뚱한 전화번호/주소 생성 방지
    - ROUGE 점수 향상 (레퍼런스와 동일한 마스킹 토큰 유지)
    """
    if not pii_masks:
        return ""
    
    if show_examples:
        examples = list(pii_masks)[:3]  # 최대 3개 예시
        examples_str = ", ".join(examples)
        return (
            f"대화 중 개인정보는 {examples_str} 와 같이 마스킹되어 있습니다. "
            "요약문에서도 이 마스킹 표현을 그대로 사용해 주세요. "
        )
    else:
        return (
            "대화 중 개인정보는 이미 마스킹되어 있습니다. "
            "요약문에서도 마스킹 표현을 그대로 유지해 주세요. "
        )

# 6. 설정
REGEX_PATTERN_CONFIG = {
    "use_speaker_count": True,      # 6-1. 화자 수 명시
    "use_pii_mask_instruction": True,  # 6-2. 마스킹 유지 지시
    "show_pii_examples": True,       # 마스킹 예시 표시 여부
}

print("\n" + "=" * 60)
print("🔍 6. 정규표현식 기반 패턴 활용 전략:")
print("=" * 60)
print(f"  6-1. 화자 수 명시: {REGEX_PATTERN_CONFIG['use_speaker_count']}")
print(f"  6-2. PII 마스킹 유지 지시: {REGEX_PATTERN_CONFIG['use_pii_mask_instruction']}")

# 예시 테스트
sample_analysis = analyze_dialogue_structure(train_df_for_tokens['dialogue'].iloc[0])
print(f"\n📊 대화 구조 분석 예시:")
print(f"  - 화자 수: {sample_analysis['num_speakers']}명")
print(f"  - 화자 토큰: {sample_analysis['speakers']}")
print(f"  - PII 마스킹 여부: {sample_analysis['has_pii_masks']}")
print(f"  - PII 마스킹 예시: {sample_analysis['pii_mask_examples']}")

# ============================================================================
# 4. 구어체 전처리: 슬랭/감탄사/의성어 정규화
# ============================================================================

# 슬랭/감탄사/의성어 → 문어체 매핑 사전
SLANG_MAP = {
    # 웃음 표현
    "ㅋㅋㅋㅋㅋ": "매우 웃기다",
    "ㅋㅋㅋㅋ": "매우 웃기다",
    "ㅋㅋㅋ": "웃기다",
    "ㅋㅋ": "웃기다",
    "ㅎㅎㅎㅎ": "웃기다",
    "ㅎㅎㅎ": "웃기다",
    "ㅎㅎ": "웃기다",
    "ㅋ": "",  # 단일 ㅋ는 제거
    "ㅎ": "",  # 단일 ㅎ는 제거
    
    # 슬픔/울음 표현
    "ㅠㅠㅠㅠ": "매우 슬프다",
    "ㅠㅠㅠ": "슬프다",
    "ㅠㅠ": "슬프다",
    "ㅜㅜㅜㅜ": "매우 슬프다",
    "ㅜㅜㅜ": "슬프다",
    "ㅜㅜ": "슬프다",
    "ㅠ": "",
    "ㅜ": "",
    
    # 놀람/감탄 표현
    "헐": "놀랍다",
    "헐ㅋ": "놀랍다",
    "대박": "놀랍다",
    "대박ㅋㅋ": "놀랍다",
    "와": "놀랍다",
    "와우": "놀랍다",
    "우와": "놀랍다",
    "오오": "놀랍다",
    "오오오": "매우 놀랍다",
    "ㅇㅇ": "그렇다",
    "ㄴㄴ": "아니다",
    
    # 긍정/동의 표현
    "ㄱㄱ": "가자",
    "ㄱㄱㄱ": "빨리 가자",
    "ㅇㅋ": "알겠다",
    "ㅇㅋㅇㅋ": "알겠다",
    "넹": "네",
    "넵": "네",
    "욤": "요",
    "용": "요",
    "쥬": "줘",
    
    # 부정/짜증 표현
    "아ㅋㅋ": "아",
    "에휴": "한숨",
    "ㅡㅡ": "짜증난다",
    ";;": "당황스럽다",
    ";;;": "매우 당황스럽다",
    
    # 강조 표현
    "진짜ㅋㅋ": "정말",
    "레알": "정말",
    "리얼": "정말",
    "찐": "진짜",
    "쩐다": "대단하다",
    "쩔어": "대단하다",
    "짱": "최고다",
    
    # 기타 인터넷 용어
    "ㄷㄷ": "놀랍다",
    "ㄷㄷㄷ": "매우 놀랍다",
    "ㅂㅂ": "안녕",
    "ㅃㅃ": "안녕",
    "ㄳ": "감사",
    "ㄱㅅ": "감사",
    "ㅈㅅ": "죄송",
    "ㅁㄹ": "모르겠다",
    "ㄹㅇ": "정말",
    "ㅇㄷ": "어디",
}

# 반복 문자 정규화 패턴
REPEAT_PATTERNS = [
    (r'ㅋ{5,}', 'ㅋㅋㅋㅋ'),
    (r'ㅎ{5,}', 'ㅎㅎㅎㅎ'),
    (r'ㅠ{5,}', 'ㅠㅠㅠㅠ'),
    (r'ㅜ{5,}', 'ㅜㅜㅜㅜ'),
    (r'\.{4,}', '...'),
    (r'!{3,}', '!!'),
    (r'\?{3,}', '??'),
    (r'~{3,}', '~~'),
]

def normalize_repeated_chars(text):
    """반복 문자를 정규화"""
    for pattern, replacement in REPEAT_PATTERNS:
        text = re.sub(pattern, replacement, text)
    return text

def normalize_slang(text):
    """슬랭/감탄사/의성어를 문어체로 변환"""
    text = normalize_repeated_chars(text)
    sorted_slang = sorted(SLANG_MAP.items(), key=lambda x: len(x[0]), reverse=True)
    for slang, replacement in sorted_slang:
        if slang in text:
            if replacement:
                text = text.replace(slang, f" {replacement} ")
            else:
                text = text.replace(slang, " ")
    text = ' '.join(text.split())
    return text

def preprocess_dialogue(text, normalize_slang_flag=True):
    """대화 전처리 파이프라인"""
    if not text or not isinstance(text, str):
        return text
    if normalize_slang_flag:
        text = normalize_slang(text)
    return text

PREPROCESS_CONFIG = {
    "normalize_slang": True,
}

print("\n" + "=" * 60)
print("🗣️ 구어체 전처리 (슬랭/감탄사 정규화):")
print("=" * 60)
print(f"총 {len(SLANG_MAP)}개 슬랭 매핑 정의")

# ============================================================================
# 5. TF-IDF 키워드 추출 및 프롬프트 활용
# ============================================================================
# (sklearn은 셀 상단에서 이미 import됨)

# TF-IDF 설정
TFIDF_CONFIG = {
    "min_df": 2,           # 최소 문서 빈도
    "max_df": 0.9,         # 최대 문서 빈도 (90% 이상 등장 단어 제외)
    "max_features": 2000,  # 최대 feature 수
    "ngram_range": (1, 2), # unigram + bigram
    "top_k": 5,            # 추출할 키워드 개수
}

print("\n" + "=" * 60)
print("📊 TF-IDF 설정:")
print("=" * 60)
print(f"  min_df: {TFIDF_CONFIG['min_df']}")
print(f"  max_df: {TFIDF_CONFIG['max_df']}")
print(f"  max_features: {TFIDF_CONFIG['max_features']}")
print(f"  ngram_range: {TFIDF_CONFIG['ngram_range']}")
print(f"  top_k: {TFIDF_CONFIG['top_k']}")

# TF-IDF Vectorizer 초기화 (한국어 불용어 포함)
KOREAN_STOPWORDS = [
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '로', '으로', '와', '과',
    '도', '만', '까지', '부터', '이다', '있다', '하다', '되다', '수', '것', '등',
    '그', '저', '이런', '저런', '그런', '어떤', '무슨', '뭐', '뭔가', '뭘',
    '나', '너', '우리', '저희', '그녀', '그', '이것', '저것', '그것',
    '아', '어', '오', '예', '네', '응', '음', '엄', '아니', '안',
    '좀', '잘', '더', '덜', '많이', '조금', '아주', '매우', '정말', '진짜',
    '그래서', '그러면', '그런데', '하지만', '그리고', '또', '또한',
    '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#',
]

class TFIDFKeywordExtractor:
    """TF-IDF 기반 키워드 추출기"""
    
    def __init__(self, corpus=None, max_features=1000, ngram_range=(1, 2)):
        """
        Args:
            corpus: TF-IDF 학습용 코퍼스 (대화 리스트)
            max_features: 최대 feature 수
            ngram_range: n-gram 범위
        """
        self.vectorizer = TfidfVectorizer(
            max_features=max_features,
            ngram_range=ngram_range,
            token_pattern=r'[가-힣]+',  # 한글만 추출
            stop_words=KOREAN_STOPWORDS,
        )
        self.is_fitted = False
        
        if corpus is not None:
            self.fit(corpus)
    
    def fit(self, corpus):
        """코퍼스로 TF-IDF 학습"""
        # 마스킹 토큰 제거 후 학습
        cleaned_corpus = [re.sub(MASK_PAT, ' ', doc) for doc in corpus]
        self.vectorizer.fit(cleaned_corpus)
        self.feature_names = self.vectorizer.get_feature_names_out()
        self.is_fitted = True
        print(f"✅ TF-IDF 학습 완료: {len(self.feature_names)}개 feature")
    
    def extract_keywords(self, text, top_k=5):
        """
        단일 문서에서 TF-IDF 상위 키워드 추출
        
        Args:
            text: 대화 텍스트
            top_k: 추출할 키워드 수
        
        Returns:
            List of (keyword, score) tuples
        """
        if not self.is_fitted:
            return []
        
        # 마스킹 토큰 제거
        cleaned_text = re.sub(MASK_PAT, ' ', text)
        
        try:
            tfidf_vector = self.vectorizer.transform([cleaned_text])
            scores = tfidf_vector.toarray()[0]
            
            # 상위 k개 인덱스
            top_indices = scores.argsort()[-top_k:][::-1]
            
            keywords = []
            for idx in top_indices:
                if scores[idx] > 0:
                    keywords.append((self.feature_names[idx], scores[idx]))
            
            return keywords
        except:
            return []
    
    def get_keyword_string(self, text, top_k=5, separator=" / "):
        """키워드를 문자열로 반환"""
        keywords = self.extract_keywords(text, top_k)
        if not keywords:
            return ""
        return separator.join([kw for kw, score in keywords])

# TF-IDF 추출기 초기화 (학습 데이터 기반)
print("\n" + "=" * 60)
print("📊 TF-IDF 키워드 추출기 초기화:")
print("=" * 60)

tfidf_extractor = TFIDFKeywordExtractor(
    corpus=train_df_for_tokens['dialogue'].tolist(),
    max_features=2000,
    ngram_range=(1, 2)
)

# 예시 테스트
sample_text = train_df_for_tokens['dialogue'].iloc[0]
sample_keywords = tfidf_extractor.get_keyword_string(sample_text, top_k=5)
print(f"\n📝 TF-IDF 키워드 추출 예시:")
print(f"  키워드: {sample_keywords}")

# ============================================================================
# 5-1. TF-IDF 키워드 기반 프롬프트 빌더 (6-1, 6-2 통합)
# ============================================================================

def build_tfidf_keyword_prompt(dialogue_text, tfidf_extractor, top_k=5, prompt_style="keyword",
                                use_speaker_count=True, use_pii_instruction=True):
    """
    TF-IDF 키워드를 포함한 프롬프트 생성 (6-1, 6-2 통합)
    
    Args:
        dialogue_text: 원본 대화 텍스트
        tfidf_extractor: TFIDFKeywordExtractor 인스턴스
        top_k: 추출할 키워드 수
        prompt_style: "keyword" (키워드만), "keyword_aware" (키워드+발화자수)
        use_speaker_count: 6-1. 화자 수 명시 여부
        use_pii_instruction: 6-2. PII 마스킹 유지 지시 여부
    
    Returns:
        프롬프트가 추가된 대화 텍스트
    """
    # 대화 구조 분석
    analysis = analyze_dialogue_structure(dialogue_text)
    num_speakers = analysis['num_speakers']
    pii_masks = analysis['pii_masks']
    
    # 키워드 추출
    keywords = tfidf_extractor.get_keyword_string(dialogue_text, top_k=top_k)
    
    prompt_parts = []
    
    # 6-1. 화자 수 명시 (keyword_aware 스타일에서만)
    if prompt_style == "keyword_aware" and use_speaker_count:
        prompt_parts.append(build_speaker_aware_prompt(num_speakers))
    elif prompt_style == "keyword":
        prompt_parts.append("다음 대화를 한 문단의 자연스러운 문어체 한국어로 요약해 주세요. ")
    
    # TF-IDF 키워드 추가
    if keywords:
        prompt_parts.append(f"주요 키워드: {keywords}. ")
    
    # 6-2. PII 마스킹 유지 지시
    if use_pii_instruction and pii_masks:
        prompt_parts.append(build_pii_mask_instruction(pii_masks, show_examples=REGEX_PATTERN_CONFIG['show_pii_examples']))
    
    prompt_parts.append("\n\n")
    
    prompt = "".join(prompt_parts)
    return prompt + dialogue_text

# ============================================================================
# 5-2. 대화 길이/주제별 동적 추론 설정
# ============================================================================

# 길이 기반 추론 설정
LENGTH_BASED_INFERENCE_CONFIG = {
    # 짧은 대화 (500자 이하)
    "short": {
        "char_threshold": 500,
        "max_new_tokens": 60,
        "num_beams": 4,
        "length_penalty": 1.0,
        "no_repeat_ngram_size": 2,
    },
    # 중간 대화 (500~1500자)
    "medium": {
        "char_threshold": 1500,
        "max_new_tokens": 80,
        "num_beams": 5,
        "length_penalty": 1.0,
        "no_repeat_ngram_size": 2,
    },
    # 긴 대화 (1500자 초과)
    "long": {
        "char_threshold": float('inf'),
        "max_new_tokens": 100,
        "num_beams": 6,
        "length_penalty": 1.2,  # 긴 대화는 요약도 길게
        "no_repeat_ngram_size": 3,
    },
}

# 주제 키워드 기반 추론 설정 (특정 주제는 더 긴 요약 필요)
TOPIC_KEYWORDS = {
    "medical": ["병원", "의사", "치료", "진료", "수술", "약", "증상", "진단", "검사"],
    "insurance": ["보험", "가입", "보장", "청구", "계약", "납입", "보상", "약관"],
    "finance": ["대출", "이자", "금리", "계좌", "송금", "입금", "출금", "투자", "주식"],
    "travel": ["여행", "비행기", "호텔", "예약", "관광", "숙소", "항공"],
}

TOPIC_INFERENCE_CONFIG = {
    "medical": {"max_new_tokens": 100, "length_penalty": 1.3},
    "insurance": {"max_new_tokens": 100, "length_penalty": 1.3},
    "finance": {"max_new_tokens": 95, "length_penalty": 1.2},
    "travel": {"max_new_tokens": 85, "length_penalty": 1.1},
    "default": {"max_new_tokens": 80, "length_penalty": 1.0},
}

def detect_topic(dialogue_text):
    """대화 주제 감지"""
    text_lower = dialogue_text.lower()
    
    topic_scores = {}
    for topic, keywords in TOPIC_KEYWORDS.items():
        score = sum(1 for kw in keywords if kw in text_lower)
        if score > 0:
            topic_scores[topic] = score
    
    if topic_scores:
        return max(topic_scores, key=topic_scores.get)
    return "default"

def get_length_category(dialogue_text):
    """대화 길이 카테고리 반환"""
    length = len(dialogue_text)
    
    if length <= LENGTH_BASED_INFERENCE_CONFIG["short"]["char_threshold"]:
        return "short"
    elif length <= LENGTH_BASED_INFERENCE_CONFIG["medium"]["char_threshold"]:
        return "medium"
    else:
        return "long"

def get_dynamic_inference_config(dialogue_text, use_topic=True, use_length=True):
    """
    대화 특성에 따른 동적 추론 설정 반환
    
    Args:
        dialogue_text: 대화 텍스트
        use_topic: 주제 기반 설정 사용 여부
        use_length: 길이 기반 설정 사용 여부
    
    Returns:
        dict: 추론 설정 (max_new_tokens, num_beams, length_penalty 등)
    """
    config = {
        "max_new_tokens": 80,
        "num_beams": 5,
        "length_penalty": 1.0,
        "no_repeat_ngram_size": 2,
    }
    
    # 1. 길이 기반 설정
    if use_length:
        length_cat = get_length_category(dialogue_text)
        length_config = LENGTH_BASED_INFERENCE_CONFIG[length_cat]
        config.update({
            "max_new_tokens": length_config["max_new_tokens"],
            "num_beams": length_config["num_beams"],
            "length_penalty": length_config["length_penalty"],
            "no_repeat_ngram_size": length_config["no_repeat_ngram_size"],
        })
    
    # 2. 주제 기반 설정 (길이보다 우선)
    if use_topic:
        topic = detect_topic(dialogue_text)
        if topic != "default":
            topic_config = TOPIC_INFERENCE_CONFIG[topic]
            config["max_new_tokens"] = max(config["max_new_tokens"], topic_config["max_new_tokens"])
            config["length_penalty"] = max(config["length_penalty"], topic_config["length_penalty"])
    
    return config

print("\n" + "=" * 60)
print("📏 길이/주제 기반 동적 추론 설정:")
print("=" * 60)
print("길이 카테고리:")
for cat, cfg in LENGTH_BASED_INFERENCE_CONFIG.items():
    print(f"  - {cat}: ~{cfg['char_threshold']}자, max_tokens={cfg['max_new_tokens']}, beams={cfg['num_beams']}")

print("\n주제 카테고리:")
for topic in TOPIC_KEYWORDS.keys():
    cfg = TOPIC_INFERENCE_CONFIG[topic]
    print(f"  - {topic}: max_tokens={cfg['max_new_tokens']}, length_penalty={cfg['length_penalty']}")

# ============================================================================
# 3-2. 프롬프트 빌더 함수 - 발화자/turn 구조 활용 (6-1, 6-2 통합)
# ============================================================================

def build_summary_prompt(dialogue_text, prompt_style="default", tfidf_extractor=None, top_k=5,
                         use_speaker_count=True, use_pii_instruction=True):
    """
    대화에 요약 프롬프트를 추가하여 모델에게 태스크를 명시적으로 알림
    6-1. 화자 수 명시: "한 사람 이야기만 요약하는" 버그 방지
    6-2. PII 마스킹 유지 지시: ROUGE 점수 향상
    
    Args:
        dialogue_text: 원본 대화 텍스트
        prompt_style: 프롬프트 스타일
            - "default": 기본 프롬프트
            - "speaker_aware": 발화자 수 포함
            - "minimal": 최소 프롬프트
            - "keyword": TF-IDF 키워드 포함 (5-1)
            - "keyword_aware": 키워드 + 발화자 수 (5-1)
            - "balanced": 6-1, 6-2 전략 적용 (NEW)
            - "none": 프롬프트 없음
        tfidf_extractor: TFIDFKeywordExtractor 인스턴스 (keyword 스타일용)
        top_k: 키워드 추출 개수
        use_speaker_count: 6-1. 화자 수 명시 여부
        use_pii_instruction: 6-2. PII 마스킹 유지 지시 여부
    
    Returns:
        프롬프트가 추가된 대화 텍스트
    """
    if prompt_style == "none":
        return dialogue_text
    
    # 대화 구조 분석 (6-1, 6-2용)
    analysis = analyze_dialogue_structure(dialogue_text)
    num_speakers = analysis['num_speakers']
    pii_masks = analysis['pii_masks']
    
    # 5-1. TF-IDF 키워드 기반 프롬프트 (6-1, 6-2 통합)
    if prompt_style in ["keyword", "keyword_aware"] and tfidf_extractor is not None:
        return build_tfidf_keyword_prompt(
            dialogue_text, tfidf_extractor, top_k, prompt_style,
            use_speaker_count=use_speaker_count,
            use_pii_instruction=use_pii_instruction
        )
    
    prompt_parts = []
    
    # 6-1. 화자 수 명시 프롬프트
    if prompt_style == "balanced":
        # 새로운 "balanced" 스타일: 6-1, 6-2 전략 모두 적용
        prompt_parts.append(build_speaker_aware_prompt(num_speakers))
    elif prompt_style == "speaker_aware":
        prompt_parts.append(build_speaker_aware_prompt(num_speakers))
    elif prompt_style == "minimal":
        prompt_parts.append("다음 대화를 요약하세요. ")
    else:  # default
        if use_speaker_count:
            prompt_parts.append(build_speaker_aware_prompt(num_speakers))
        else:
            prompt_parts.append("다음은 여러 사람이 나눈 일상 대화입니다. 이 대화를 한 문단의 자연스러운 문어체 한국어로 요약해 주세요. ")
    
    # 6-2. PII 마스킹 유지 지시 (balanced, default, speaker_aware 스타일에서)
    if use_pii_instruction and pii_masks and prompt_style in ["balanced", "default", "speaker_aware"]:
        prompt_parts.append(build_pii_mask_instruction(pii_masks, show_examples=REGEX_PATTERN_CONFIG['show_pii_examples']))
    
    prompt_parts.append("\n\n")
    
    prompt = "".join(prompt_parts)
    return prompt + dialogue_text

def add_turn_separators(dialogue_text, separator="[SEP]"):
    """발화 turn 사이에 구분자 추가"""
    lines = dialogue_text.strip().split('\n')
    result_lines = []
    for i, line in enumerate(lines):
        if i > 0 and re.match(PERSON_PAT, line.strip()):
            result_lines.append(f"{separator} {line}")
        else:
            result_lines.append(line)
    return '\n'.join(result_lines)

# ============================================================================
# 설정 업데이트 (6. 정규표현식 패턴 활용 전략 반영)
# ============================================================================

config_data = {
    "general": {
        "data_path": "./data/",
        "model_name": "digit82/kobart-summarization",
        "output_dir": "./prediction_kobart_v1"
    },
    "tokenizer": {
        "encoder_max_len": 1024,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "special_tokens": DYNAMIC_SPECIAL_TOKENS
    },
    # 3-2. 프롬프트 설정 (6-1, 6-2 통합)
    "prompt": {
        "use_prompt": True,
        "prompt_style": "balanced",  # NEW: 6-1, 6-2 전략 적용
        "use_turn_separator": False,
        "turn_separator": "[SEP]",
        "tfidf_top_k": 5,  # 5-1. TF-IDF 키워드 개수
    },
    # 6. 정규표현식 패턴 활용 설정 (NEW)
    "regex_pattern": {
        "use_speaker_count": True,       # 6-1. 화자 수 명시
        "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
        "show_pii_examples": True,       # 마스킹 예시 표시
    },
    # 4. 구어체 전처리 설정
    "preprocess": {
        "normalize_slang": True,
    },
    # 5-2. 동적 추론 설정
    "dynamic_inference": {
        "use_length_based": True,   # 길이 기반 동적 설정 사용
        "use_topic_based": True,    # 주제 기반 동적 설정 사용
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 16,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "eval_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb"
    },
    "wandb": {
        "entity": "fc_bootcamp",
        "project": "kobart_v1",
        "name": "baseline_v0"
    },
    "inference": {
        "ckt_path": "model ckt path",
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size": 32,
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

# ============================================================================
# 설정 요약 출력
# ============================================================================

print("\n" + "=" * 60)
print("📋 전체 설정 요약:")
print("=" * 60)

print("\n🔖 3-1. Special Tokens:")
print(f"  - 동적 수집: {len(DYNAMIC_SPECIAL_TOKENS)}개")

print("\n📝 3-2. 프롬프트 설정:")
print(f"  - 프롬프트 사용: {config_data['prompt']['use_prompt']}")
print(f"  - 프롬프트 스타일: {config_data['prompt']['prompt_style']}")
print(f"  - TF-IDF 키워드 수: {config_data['prompt']['tfidf_top_k']}개")

print("\n🔍 6. 정규표현식 패턴 활용 전략:")
print(f"  - 6-1. 화자 수 명시: {config_data['regex_pattern']['use_speaker_count']}")
print(f"  - 6-2. PII 마스킹 유지 지시: {config_data['regex_pattern']['use_pii_instruction']}")

print("\n🗣️ 4. 구어체 전처리:")
print(f"  - 슬랭 정규화: {config_data['preprocess']['normalize_slang']}")

print("\n📏 5-2. 동적 추론:")
print(f"  - 길이 기반: {config_data['dynamic_inference']['use_length_based']}")
print(f"  - 주제 기반: {config_data['dynamic_inference']['use_topic_based']}")

# ============================================================================
# 6. 전처리 파이프라인 예시 (6-1, 6-2 통합)
# ============================================================================
print("\n" + "-" * 60)
print("📝 전처리 파이프라인 예시 (6-1, 6-2 적용):")
print("-" * 60)

sample_dialogue = train_df_for_tokens['dialogue'].iloc[0]
print(f"[1. 원본 대화 (앞 150자)]:\n{sample_dialogue[:150]}...")

# 대화 구조 분석
sample_analysis = analyze_dialogue_structure(sample_dialogue)
print(f"\n[2. 대화 구조 분석]:")
print(f"  - 화자 수: {sample_analysis['num_speakers']}명")
print(f"  - PII 마스킹 토큰: {sample_analysis['pii_mask_examples']}")

sample_preprocessed = preprocess_dialogue(sample_dialogue, normalize_slang_flag=True)
print(f"\n[3. 슬랭 정규화 후]:\n{sample_preprocessed[:150]}...")

sample_keywords = tfidf_extractor.get_keyword_string(sample_preprocessed, top_k=5)
print(f"\n[4. TF-IDF 키워드]: {sample_keywords}")

# 6-1, 6-2 통합 프롬프트 (balanced 스타일)
sample_prompted = build_summary_prompt(
    sample_preprocessed, 
    prompt_style="balanced",
    use_speaker_count=True,
    use_pii_instruction=True
)
print(f"\n[5. 'balanced' 프롬프트 적용 (6-1, 6-2)]:\n{sample_prompted[:400]}...")

# keyword_aware 스타일 예시
sample_keyword_aware = build_summary_prompt(
    sample_preprocessed,
    prompt_style="keyword_aware",
    tfidf_extractor=tfidf_extractor,
    top_k=5,
    use_speaker_count=True,
    use_pii_instruction=True
)
print(f"\n[6. 'keyword_aware' 프롬프트 적용 (TF-IDF + 6-1, 6-2)]:\n{sample_keyword_aware[:400]}...")

# 동적 추론 설정 예시
dynamic_config = get_dynamic_inference_config(sample_dialogue, use_topic=True, use_length=True)
print(f"\n[7. 동적 추론 설정]:")
print(f"  - 길이 카테고리: {get_length_category(sample_dialogue)}")
print(f"  - 감지된 주제: {detect_topic(sample_dialogue)}")
print(f"  - max_new_tokens: {dynamic_config['max_new_tokens']}")

# ============================================================================
# 유틸리티 함수: 디코딩 & 후처리
# ============================================================================

def decode_with_speaker_tags(tokenizer, output_ids, remove_tokens=None):
    """
    화자 태그(#Person1# 등)를 보존하면서 디코딩
    skip_special_tokens=False로 디코딩 후, 불필요한 토큰만 제거
    """
    if remove_tokens is None:
        remove_tokens = ['<usr>', tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token]
    
    # skip_special_tokens=False로 디코딩 (화자 태그 보존)
    text = tokenizer.decode(output_ids, skip_special_tokens=False)
    
    # 불필요한 토큰만 제거
    for token in remove_tokens:
        if token:
            text = text.replace(token, '')
    
    return text.strip()

def postprocess_summary(summary):
    """
    요약문 후처리:
    1. 불필요한 공백 제거
    2. 화자 태그 앞뒤 공백 정리
    """
    import re
    
    # 연속 공백 제거
    summary = ' '.join(summary.split())
    
    # 화자 태그 앞뒤 공백 정리 (#Person1# 은 → #Person1#은)
    summary = re.sub(r'(#Person\d+#)\s+', r'\1 ', summary)
    
    return summary.strip()

print("\n✅ decode_with_speaker_tags, postprocess_summary 함수 정의 완료")

🔖 동적으로 수집된 Special Tokens:
총 22개: ['#Address#', '#Alex#', '#Bob#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#Kristin#', '#Liliana#', '#Name#', '#PassportNumber#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PersonName#', '#PhoneNumber#', '#Price#', '#SSN#']

🔍 6. 정규표현식 기반 패턴 활용 전략:
  6-1. 화자 수 명시: True
  6-2. PII 마스킹 유지 지시: True

📊 대화 구조 분석 예시:
  - 화자 수: 2명
  - 화자 토큰: {'#Person1#', '#Person2#'}
  - PII 마스킹 여부: False
  - PII 마스킹 예시: []

🗣️ 구어체 전처리 (슬랭/감탄사 정규화):
총 59개 슬랭 매핑 정의

📊 TF-IDF 키워드 추출기 초기화:
✅ TF-IDF 학습 완료: 2000개 feature

📝 TF-IDF 키워드 추출 예시:
  키워드: 매년 / 한 / 번 / 해요 / 의사 선생님

📏 길이/주제 기반 동적 추론 설정:
길이 카테고리:
  - short: ~500자, max_tokens=60, beams=4
  - medium: ~1500자, max_tokens=80, beams=5
  - long: ~inf자, max_tokens=100, beams=6

주제 카테고리:
  - medical: max_tokens=100, length_penalty=1.3
  - insurance: max_tokens=100, length_penalty=1.3
  - finance: max_tokens=95, length_penalty=1.2
  - travel: max_tokens=85, length_p

- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

In [4]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 3) Configuration 불러오기

In [5]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'dynamic_inference': {'use_length_based': True, 'use_topic_based': True},
 'general': {'data_path': './data/',
             'model_name': 'digit82/kobart-summarization',
             'output_dir': './prediction_kobart_v1'},
 'inference': {'batch_size': 32,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
               'result_path': './prediction/'},
 'preprocess': {'normalize_slang': True},
 'prompt': {'prompt_style': 'balanced',
            'tfidf_top_k': 5,
            'turn_separator': '[SEP]',
            'use_prompt': True,
            'use_turn_separator': False},
 'regex_pattern': {'show_pii_examples': True,
                   'use_pii_instruction': True,
                   'use_speaker_count': True},
 'tokenizer': {'bos_token': '<s>',
               'decoder_ma

In [6]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

{'data_path': './data/',
 'model_name': 'digit82/kobart-summarization',
 'output_dir': './prediction_kobart_v1'}

In [7]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
# loaded_config['general']['data_path'] = "data_path"

# Ensure compatibility with Transformers: they expect the key to be 'evaluation_strategy'.
# Some configs may use 'eval_strategy' (short form). Normalize to 'evaluation_strategy'.
if 'eval_strategy' in loaded_config['training'] and 'evaluation_strategy' not in loaded_config['training']:
    loaded_config['training']['evaluation_strategy'] = loaded_config['training'].pop('eval_strategy')

# (legacy) if someone used 'evaluation_strategy' already, keep it as-is.


In [8]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'bos_token': '<s>',
 'decoder_max_len': 100,
 'encoder_max_len': 1024,
 'eos_token': '</s>',
 'special_tokens': ['#Address#',
  '#Alex#',
  '#Bob#',
  '#CarNumber#',
  '#CardNumber#',
  '#DateOfBirth#',
  '#Email#',
  '#Kristin#',
  '#Liliana#',
  '#Name#',
  '#PassportNumber#',
  '#Person1#',
  '#Person2#',
  '#Person3#',
  '#Person4#',
  '#Person5#',
  '#Person6#',
  '#Person7#',
  '#PersonName#',
  '#PhoneNumber#',
  '#Price#',
  '#SSN#']}

In [9]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'fp16': True,
 'generation_max_length': 100,
 'gradient_accumulation_steps': 1,
 'learning_rate': 2e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 32,
 'per_device_train_batch_size': 16,
 'predict_with_generate': True,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01,
 'evaluation_strategy': 'epoch'}

In [10]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
loaded_config['wandb']

{'entity': 'fc_bootcamp', 'name': 'baseline_v0', 'project': 'kobart_v1'}

In [11]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
loaded_config['wandb']['entity'] = "fc_bootcamp"
loaded_config['wandb']['project'] = "kobart_v1"
loaded_config['wandb']['name'] = "baseline_v0"

# wandb 비활성화 (권한 문제 해결)
loaded_config['training']['report_to'] = "wandb"

In [12]:
# --- Wandb setup helper (추가) ---
# 사용법 예시:
# cfg = setup_wandb(api_key="<YOUR_API_KEY>", entity="your-entity", project="your-project", run_name="exp-01")
# import wandb
# wandb.init(**cfg)

import os

def setup_wandb(api_key: str = None, entity: str = None, project: str = None, run_name: str = None, mode: str = 'online'):
    """
    Helper to set WANDB API key and login programmatically.

    - api_key: your W&B API key (string). If provided, sets `WANDB_API_KEY` env var.
    - entity: W&B team or user name (maps to `entity` in `wandb.init`).
    - project: W&B project name (maps to `project` in `wandb.init`).
    - run_name: a human-friendly run name (maps to `name` in `wandb.init`).
    - mode: 'online' or 'offline' (useful when running without internet or CI).

    Returns a dict that can be passed to `wandb.init(**cfg)` or used to populate `loaded_config['wandb']`.
    """
    if api_key:
        os.environ['WANDB_API_KEY'] = '624410f236117b79b2ee07a4ccef31c7514a5e03'

    try:
        import wandb
    except Exception as e:
        raise ImportError("wandb is not installed. Install with `pip install wandb` and try again.") from e

    env_key = os.environ.get('WANDB_API_KEY')
    if env_key is None:
        print("Warning: No WANDB_API_KEY found in environment. Call setup_wandb(api_key=...) or set WANDB_API_KEY manually.")

    # Perform login (this will use the env var if present)
    try:
        wandb.login(key=env_key)
    except Exception as e:
        print("wandb.login() failed:", e)

    cfg = {
        'mode': mode
    }
    if entity:
        cfg['entity'] = fc_bootcamp
    if project:
        cfg['project'] = kobart_v1
    if run_name:
        cfg['name'] = baseline_v0

    print("W&B helper ready. Use: import wandb; wandb.init(**cfg)")
    return cfg

# 예시 (주석 해제 후 사용):
# cfg = setup_wandb(api_key="<YOUR_API_KEY>", entity="your-team-or-username", project="my-project", run_name="run-1")
# import wandb
# wandb.init(**cfg)
# ----------------------------------


In [13]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 32,
 'ckt_path': 'model ckt path',
 'early_stopping': True,
 'generate_max_length': 100,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
 'result_path': './prediction/'}

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [14]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 안녕하세요. 혹시 맨체스터에서 오신 Mr. Green 맞으신가요...,Tan Ling은 흰머리와 수염이 특징인 Mr. Green을 맞이하여 호텔로 안내합...,호텔 안내
12453,train_12456,"#Person1#: Mister Ewing이 우리 회의장에 4시에 오라고 했지, 맞...",#Person1#과 #Person2#는 Mister Ewing의 요청에 따라 회의장...,회의 준비
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형차를 대여합니다.,차량 대여
12455,train_12458,#Person1#: 너 오늘 좀 기분 안 좋아 보인다? 무슨 일 있어?\n#Pers...,#Person2#의 어머니가 직장을 잃으셨다. #Person2#는 어머니가 우울해하...,실직과 대처
12456,train_12459,"#Person1#: 엄마, 나 다음 주 토요일에 이모부네 가족 보러 가는데, 오늘 ...",#Person1#은 다음 주 토요일에 이모부네 가족을 방문하기 위해 짐을 싸야 하는...,가족 방문 준비


In [15]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 새해가 되니까 나도 새 출발을 하기로 했어.\n#Person2#...,#Person1#은 새해에 담배를 끊고 커밍아웃 하기로 결심했습니다. #Person...,새해 결심
495,dev_496,#Person1#: 너 Joe랑 결혼했지?\n#Person2#: Joe? 무슨 말이...,"#Person1#은 #Person2#가 Joe와 결혼했다고 생각하지만, #Perso...",사랑과 결혼 오해
496,dev_497,"#Person1#: 어떻게 도와드릴까요, 아줌마?\n#Person2#: 제 차에서 ...","#Person2#의 차에서 소리가 나며, 브레이크 수리가 필요한 상황입니다. #Pe...",차량 소음 및 수리
497,dev_498,"#Person1#: 여보세요, 아마존 고객 서비스입니다. 어떻게 도와드릴까요?\n#...",#Person2#가 아마존 고객 서비스에 전화하여 아마존에서 구매한 책에 53페이지...,책 페이지 누락
498,dev_499,#Person1#: 벌써 여름이 다가오다니 믿기지 않아. \n#Person2#: 맞...,"#Person2#는 여름방학 동안 파티에서 일하는 회사에서 일하며, 주로 음식 준비...",여름방학 일자리


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [16]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
            prompt_config: dict = None,           # 3-2. 프롬프트 설정
            preprocess_config: dict = None,       # 4. 구어체 전처리 설정
            tfidf_extractor = None,               # 5-1. TF-IDF 키워드 추출기
            regex_pattern_config: dict = None,    # 6. 정규표현식 패턴 활용 설정 (NEW)
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token
        
        # 프롬프트 설정 저장
        self.prompt_config = prompt_config or {
            "use_prompt": False,
            "prompt_style": "none",
            "use_turn_separator": False,
            "turn_separator": "[SEP]",
            "tfidf_top_k": 5,
        }
        
        # 4. 구어체 전처리 설정 저장
        self.preprocess_config = preprocess_config or {
            "normalize_slang": False,
        }
        
        # 5-1. TF-IDF 키워드 추출기
        self.tfidf_extractor = tfidf_extractor
        
        # 6. 정규표현식 패턴 활용 설정 (NEW)
        self.regex_pattern_config = regex_pattern_config or {
            "use_speaker_count": True,       # 6-1. 화자 수 명시
            "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
            "show_pii_examples": True,       # 마스킹 예시 표시
        }

    @staticmethod
    def make_set_as_df(file_path, is_train=True):
        """실험에 필요한 컬럼을 가져옵니다."""
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname', 'dialogue', 'summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname', 'dialogue']]
            return test_df

    def apply_slang_normalization(self, text):
        """4. 구어체 전처리: 슬랭/감탄사/의성어 정규화"""
        if self.preprocess_config.get("normalize_slang", False):
            return preprocess_dialogue(text, normalize_slang_flag=True)
        return text

    def apply_prompt_to_dialogue(self, dialogue_text):
        """
        3-2, 5-1, 6-1, 6-2 통합 프롬프트 적용
        
        - 3-2: 프롬프트 스타일 적용
        - 5-1: TF-IDF 키워드 포함
        - 6-1: 화자 수 명시 (균형 있는 요약 유도)
        - 6-2: PII 마스킹 유지 지시 (ROUGE 점수 향상)
        """
        text = dialogue_text
        
        # 발화 구분자 적용 (선택적)
        if self.prompt_config.get("use_turn_separator", False):
            text = add_turn_separators(text, self.prompt_config.get("turn_separator", "[SEP]"))
        
        # 프롬프트 적용
        if self.prompt_config.get("use_prompt", False):
            prompt_style = self.prompt_config.get("prompt_style", "default")
            tfidf_top_k = self.prompt_config.get("tfidf_top_k", 5)
            
            # 6-1, 6-2 설정 가져오기
            use_speaker_count = self.regex_pattern_config.get("use_speaker_count", True)
            use_pii_instruction = self.regex_pattern_config.get("use_pii_instruction", True)
            
            # 5-1. TF-IDF 키워드 스타일인 경우 extractor 전달
            if prompt_style in ["keyword", "keyword_aware"] and self.tfidf_extractor is not None:
                text = build_summary_prompt(
                    text, 
                    prompt_style=prompt_style, 
                    tfidf_extractor=self.tfidf_extractor, 
                    top_k=tfidf_top_k,
                    use_speaker_count=use_speaker_count,      # 6-1
                    use_pii_instruction=use_pii_instruction    # 6-2
                )
            else:
                # 기타 스타일에서도 6-1, 6-2 적용
                text = build_summary_prompt(
                    text, 
                    prompt_style=prompt_style,
                    use_speaker_count=use_speaker_count,      # 6-1
                    use_pii_instruction=use_pii_instruction    # 6-2
                )
        
        return text

    def full_preprocess_pipeline(self, dialogue_text):
        """
        전체 전처리 파이프라인: 슬랭 정규화 → 프롬프트 적용
        
        처리 순서:
        1. 슬랭/감탄사/의성어 정규화 (4. 구어체 전처리)
        2. 발화 구분자 추가 (선택적)
        3. 프롬프트 추가 (3-2. 프롬프트 설정)
           - TF-IDF 키워드 포함 (5-1)
           - 화자 수 명시 (6-1)
           - PII 마스킹 유지 지시 (6-2)
        """
        text = dialogue_text
        
        # Step 1: 슬랭 정규화
        text = self.apply_slang_normalization(text)
        
        # Step 2 & 3: 프롬프트 적용 (구분자 + TF-IDF + 6-1 + 6-2)
        text = self.apply_prompt_to_dialogue(text)
        
        return text

    def make_input(self, dataset, is_test=False, tokenizer=None, encoder_max_len=None, 
                   prompt=None, apply_dynamic_prompt=True):
        """
        BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
        
        전처리 파이프라인:
        1. 슬랭/감탄사 정규화 (4. 구어체 전처리)
        2. TF-IDF 키워드 추출 (5-1)
        3. 프롬프트 추가 (3-2. 프롬프트 설정)
        4. 6-1 화자 수 명시 + 6-2 PII 마스킹 유지 지시
        5. 토큰 기반 truncation (선택적)
        """
        if is_test:
            encoder_input = []
            decoder_input = []
            for d in dataset['dialogue']:
                text = d
                
                # 전체 전처리 파이프라인 적용
                if apply_dynamic_prompt:
                    text = self.full_preprocess_pipeline(text)
                elif prompt:
                    text = self.apply_slang_normalization(text)
                    text = apply_summary_prompt(text, prompt)
                else:
                    text = self.apply_slang_normalization(text)
                
                # 토큰 기반 truncation
                if tokenizer is not None and encoder_max_len is not None:
                    text = smart_truncate_dialogue(text, tokenizer, encoder_max_len)
                
                encoder_input.append(text)
                decoder_input.append(self.bos_token)
            return encoder_input, decoder_input
        else:
            # 학습 데이터 처리
            encoder_input = []
            for d in dataset['dialogue']:
                text = d
                
                # 전체 전처리 파이프라인 적용
                if apply_dynamic_prompt:
                    text = self.full_preprocess_pipeline(text)
                else:
                    text = self.apply_slang_normalization(text)
                
                encoder_input.append(text)
            
            # Gold target을 디코더 입력으로 사용합니다.
            decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x))
            decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)
            return encoder_input, decoder_input.tolist(), decoder_output.tolist()

In [17]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len


In [18]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    """
    3-2 개선: 프롬프트 설정이 적용된 데이터셋 준비
    """
    train_file_path = os.path.join(data_path,'train.csv')
    val_file_path = os.path.join(data_path,'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*150)
    print(f'[원본] train_data dialogue:\n {train_data["dialogue"][0][:200]}...')
    print(f'train_label:\n {train_data["summary"][0]}')

    # 3-2. 프롬프트 적용된 데이터로 변환
    encoder_input_train, decoder_input_train, decoder_output_train = preprocessor.make_input(
        train_data, 
        apply_dynamic_prompt=True  # 프롬프트 설정 사용
    )
    encoder_input_val, decoder_input_val, decoder_output_val = preprocessor.make_input(
        val_data,
        apply_dynamic_prompt=True
    )
    
    # 프롬프트 적용 확인
    print('-'*150)
    print(f'[프롬프트 적용 후] encoder_input_train[0]:\n {encoder_input_train[0][:300]}...')
    print('-'*10, 'Load data complete', '-'*10)

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10)
    
    # 토큰화 후 special token 확인
    print("\n📊 토큰화 결과 확인:")
    sample_tokens = tokenized_encoder_inputs['input_ids'][0][:50]
    decoded_sample = tokenizer.decode(sample_tokens)
    print(f"   첫 50 토큰 디코딩: {decoded_sample}...")
    
    return train_inputs_dataset, val_inputs_dataset

## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [19]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [20]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result


# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training'].get('evaluation_strategy', 'epoch'), # transformers expects this key
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )

    # (선택) wandb 사용 시에만 초기화
    if config['training']['report_to'] == 'wandb':
        wandb.init(
            entity=config['wandb']['entity'],
            project=config['wandb']['project'],
            name=config['wandb']['name'],
        )
        # 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
        os.environ["WANDB_LOG_MODEL"]="true"
        os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [21]:
# ============================================================================
# 3-1. 토크나이저 및 모델 로드 (Special Token 동적 등록 포함)
# ============================================================================

def load_tokenizer_and_model_for_train(config, device, verbose=True):
    """
    학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
    
    3-1 개선사항:
    - 학습 데이터에서 동적으로 수집한 special tokens 등록
    - model.resize_token_embeddings() 호출로 임베딩 사이즈 조정
    """
    if verbose:
        print('-' * 10, 'Load tokenizer & model', '-' * 10)
        print('-' * 10, f'Model Name : {config["general"]["model_name"]}', '-' * 10)
    
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(
        config['general']['model_name'], 
        config=bart_config
    )

    # ============================================================
    # 3-1. Special Tokens 등록 (동적으로 수집된 토큰 사용)
    # ============================================================
    special_tokens = config['tokenizer']['special_tokens']
    
    if verbose:
        print(f"\n📌 등록할 Special Tokens ({len(special_tokens)}개):")
        print(f"   {special_tokens}")
    
    # additional_special_tokens로 등록
    special_tokens_dict = {'additional_special_tokens': special_tokens}
    num_added = tokenizer.add_special_tokens(special_tokens_dict)
    
    if verbose:
        print(f"\n✅ 새로 추가된 토큰 수: {num_added}")
        print(f"   토크나이저 vocab size: {len(tokenizer)}")

    # ⚠️ 중요: 모델 임베딩 사이즈 조정
    generate_model.resize_token_embeddings(len(tokenizer))
    
    if verbose:
        print(f"   모델 임베딩 사이즈 조정 완료: {generate_model.config.vocab_size}")
    
    generate_model.to(device)
    
    if verbose:
        print("\n" + "-" * 60)
        print("🔍 Special Token 확인:")
        print("-" * 60)
        for token in special_tokens[:5]:  # 처음 5개만 출력
            token_id = tokenizer.convert_tokens_to_ids(token)
            print(f"   {token} → ID: {token_id}")
        if len(special_tokens) > 5:
            print(f"   ... 외 {len(special_tokens) - 5}개")
        
        print('-' * 10, 'Load tokenizer & model complete', '-' * 10)
    
    return generate_model, tokenizer


def load_tokenizer_and_model_for_inference(config, checkpoint_path, device, verbose=True):
    """
    추론을 위한 tokenizer와 fine-tuned 모델을 불러옵니다.
    
    3-1 개선사항:
    - checkpoint에서 로드 시에도 special tokens 일관성 유지
    """
    if verbose:
        print('-' * 10, 'Load tokenizer & model for inference', '-' * 10)
        print('-' * 10, f'Checkpoint: {checkpoint_path}', '-' * 10)
    
    # Checkpoint에서 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    
    # Special tokens 확인 및 추가 (checkpoint에 없을 경우)
    special_tokens = config['tokenizer']['special_tokens']
    existing_special = tokenizer.additional_special_tokens or []
    
    missing_tokens = [t for t in special_tokens if t not in existing_special]
    if missing_tokens:
        if verbose:
            print(f"⚠️ 누락된 special tokens 추가: {missing_tokens}")
        special_tokens_dict = {'additional_special_tokens': special_tokens}
        tokenizer.add_special_tokens(special_tokens_dict)
    
    # 모델 로드
    model = BartForConditionalGeneration.from_pretrained(checkpoint_path)
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    model.eval()
    
    if verbose:
        print(f"✅ 모델 로드 완료")
        print(f"   토크나이저 vocab size: {len(tokenizer)}")
        print(f"   모델 임베딩 사이즈: {model.config.vocab_size}")
        print('-' * 10, 'Load complete', '-' * 10)
    
    return model, tokenizer

## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [22]:
def main(config):
    """
    모델 학습 메인 함수
    3-1, 3-2, 4, 5-1, 6-1, 6-2 개선: 
    - Special tokens + 프롬프트 + 구어체 전처리 + TF-IDF 키워드
    - 화자 수 명시 + PII 마스킹 유지 지시
    """
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-' * 10, f'device : {device}', '-' * 10)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model, tokenizer = load_tokenizer_and_model_for_train(config, device)
    print('-' * 10, "tokenizer special tokens : ", tokenizer.special_tokens_map, '-' * 10)

    # 3-2. 프롬프트 설정
    prompt_config = config.get('prompt', {
        "use_prompt": False,
        "prompt_style": "none",
        "use_turn_separator": False,
        "turn_separator": "[SEP]",
        "tfidf_top_k": 5,
    })
    
    # 4. 구어체 전처리 설정
    preprocess_config = config.get('preprocess', {
        "normalize_slang": False,
    })
    
    # 6. 정규표현식 패턴 활용 설정 (NEW)
    regex_pattern_config = config.get('regex_pattern', {
        "use_speaker_count": True,       # 6-1. 화자 수 명시
        "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
        "show_pii_examples": True,       # 마스킹 예시 표시
    })
    
    print(f"\n📋 프롬프트 설정:")
    print(f"   - 프롬프트 사용: {prompt_config.get('use_prompt', False)}")
    print(f"   - 프롬프트 스타일: {prompt_config.get('prompt_style', 'none')}")
    print(f"   - 발화 구분자 사용: {prompt_config.get('use_turn_separator', False)}")
    if prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']:
        print(f"   - TF-IDF 키워드 수: {prompt_config.get('tfidf_top_k', 5)}개")
    
    print(f"\n🗣️ 구어체 전처리 설정:")
    print(f"   - 슬랭 정규화: {preprocess_config.get('normalize_slang', False)}")
    
    print(f"\n🔍 6. 정규표현식 패턴 활용:")
    print(f"   - 6-1. 화자 수 명시: {regex_pattern_config.get('use_speaker_count', True)}")
    print(f"   - 6-2. PII 마스킹 유지 지시: {regex_pattern_config.get('use_pii_instruction', True)}")
    
    # 5-1. TF-IDF extractor 사용 여부 확인
    use_tfidf = prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']
    tfidf_ext = tfidf_extractor if use_tfidf else None
    
    if use_tfidf:
        print(f"\n📊 TF-IDF 키워드 추출 활성화")
    
    preprocessor = Preprocess(
        config['tokenizer']['bos_token'], 
        config['tokenizer']['eos_token'],
        prompt_config=prompt_config,
        preprocess_config=preprocess_config,
        tfidf_extractor=tfidf_ext,           # 5-1. TF-IDF extractor 전달
        regex_pattern_config=regex_pattern_config  # 6. 정규표현식 패턴 설정 전달
    )
    
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config, preprocessor, data_path, tokenizer)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) wandb 사용 시에만 종료
    if config['training']['report_to'] == 'wandb':
        wandb.finish()

In [23]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



📌 등록할 Special Tokens (22개):
   ['#Address#', '#Alex#', '#Bob#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#Kristin#', '#Liliana#', '#Name#', '#PassportNumber#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PersonName#', '#PhoneNumber#', '#Price#', '#SSN#']

✅ 새로 추가된 토큰 수: 22
   토크나이저 vocab size: 30022
   모델 임베딩 사이즈 조정 완료: 30022

------------------------------------------------------------
🔍 Special Token 확인:
------------------------------------------------------------
   #Address# → ID: 30002
   #Alex# → ID: 30007
   #Bob# → ID: 30008
   #CarNumber# → ID: 30006
   #CardNumber# → ID: 30020
   ... 외 17개
---------- Load tokenizer & model complete ----------
---------- tokenizer special tokens :  {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>', 'additional_special_tokens': ['#Email#', '#Name#', '#Address#', '#Person7#', '#Person3#', '#PhoneNumber#', '#CarNumber#',

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


---------- Make dataset complete ----------

📊 토큰화 결과 확인:
   첫 50 토큰 디코딩: 다음은 2명이 나눈 대화입니다. 두 사람의 입장을 균형 있게 반영해서 요약해 주세요. 

#Person1# : 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? ...
---------- Make training arguments ----------


...

Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,2.825000,0.566371,0.344045,0.109752,0.325590
2,0.570300,0.525475,0.363801,0.128972,0.340726
3,0.495200,0.510620,0.366163,0.130008,0.344374
4,0.430900,0.509380,0.369940,0.139470,0.345888
5,0.378400,0.517669,0.366900,0.136874,0.344868
6,0.334300,0.528289,0.373542,0.140957,0.351235
7,0.295500,0.543136,0.371000,0.140288,0.347263


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 #Person2# 에게 감기에 걸렸으며, 천식 검사를 위해 폐 전문의를 방문하라고 권장합니다.                                                                            
GOLD: #Person2# 는 숨쉬기 어려워합니다. 의사는 #Person2# 에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.                                                                     
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 야 Jimmy에게 운동하러 가자고 제안합니다. 야는 토요일에 운동할 예정입니다.                                                                             
GOLD: #Person1# 는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.                                                                                
---------------------------------------------------------------------------------------------------------------------

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/loss,█▃▁▁▂▃▅
eval/rouge-1,▁▆▆▇▆█▇
eval/rouge-2,▁▅▆█▇██
eval/rouge-l,▁▅▆▇▆█▇
eval/runtime,▄▁▄▄▂▃█
eval/samples_per_second,▅█▅▅▇▆▁
eval/steps_per_second,▅█▅▅▇▆▁
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,▁███▇▆▆
train/loss,█▂▂▁▁▁▁


## 4. 모델 추론하기

In [24]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "./prediction_kobart_v1/checkpoint-4674"

- test data를 사용하여 모델의 성능을 확인합니다.

In [25]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config, preprocessor, tokenizer):
    """
    3-2 개선: 프롬프트 설정이 적용된 테스트 데이터셋 준비
    """
    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'[원본] test_data dialogue:\n{test_data["dialogue"][0][:200]}...')
    print('-'*150)

    # 3-2. 프롬프트 적용된 테스트 데이터로 변환
    encoder_input_test, decoder_input_test = preprocessor.make_input(
        test_data, 
        is_test=True,
        apply_dynamic_prompt=True  # 프롬프트 설정 사용
    )
    
    # 프롬프트 적용 확인
    print(f'[프롬프트 적용 후] encoder_input_test[0]:\n{encoder_input_test[0][:300]}...')
    print('-'*10, 'Load data complete', '-'*10)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10)

    return test_data, test_encoder_inputs_dataset

In [26]:
# ============================================================================
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
# 3-1 개선: 동적으로 수집된 special tokens 사용
# ============================================================================

def load_tokenizer_and_model_for_test(config, device, verbose=True):
    """
    추론용 토크나이저 및 모델 로드
    3-1: 동적으로 수집된 special tokens 등록
    """
    if verbose:
        print('-' * 10, 'Load tokenizer & model', '-' * 10)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    
    if verbose:
        print('-' * 10, f'Model Name : {model_name}', '-' * 10)
        print('-' * 10, f'Checkpoint : {ckt_path}', '-' * 10)
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # 3-1. 동적으로 수집된 special tokens 등록
    special_tokens = config['tokenizer']['special_tokens']
    special_tokens_dict = {'additional_special_tokens': special_tokens}
    num_added = tokenizer.add_special_tokens(special_tokens_dict)
    
    if verbose:
        print(f"\n📌 등록된 Special Tokens ({len(special_tokens)}개)")
        print(f"   새로 추가된 토큰: {num_added}개")

    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    
    if verbose:
        print(f"   토크나이저 vocab size: {len(tokenizer)}")
        print(f"   모델 임베딩 사이즈: {generate_model.config.vocab_size}")
        print('-' * 10, 'Load tokenizer & model complete', '-' * 10)

    return generate_model, tokenizer

In [27]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    """
    모델 추론 함수
    3-1, 3-2, 4, 5-1, 5-2, 6-1, 6-2 개선: 
    - Special tokens 동적 등록
    - 프롬프트 설정
    - 구어체 전처리 적용
    - TF-IDF 키워드 프롬프트
    - 동적 추론 설정 (길이 기반)
    - 화자 수 명시 (6-1)
    - PII 마스킹 유지 지시 (6-2)
    """
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-' * 10, f'device : {device}', '-' * 10)
    print(torch.__version__)

    generate_model, tokenizer = load_tokenizer_and_model_for_test(config, device)

    # 3-2. 프롬프트 설정
    prompt_config = config.get('prompt', {
        "use_prompt": False,
        "prompt_style": "none",
        "use_turn_separator": False,
        "turn_separator": "[SEP]",
        "tfidf_top_k": 5,
    })
    
    # 4. 구어체 전처리 설정
    preprocess_config = config.get('preprocess', {
        "normalize_slang": False,
    })
    
    # 6. 정규표현식 패턴 활용 설정 (NEW)
    regex_pattern_config = config.get('regex_pattern', {
        "use_speaker_count": True,       # 6-1. 화자 수 명시
        "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
        "show_pii_examples": True,       # 마스킹 예시 표시
    })
    
    # 5-2. 동적 추론 설정 (길이 기반)
    dynamic_inference_config = config.get('dynamic_inference', {})
    use_dynamic = dynamic_inference_config.get('use_length_based', False) or dynamic_inference_config.get('use_dynamic_settings', False)
    
    print(f"\n📋 추론 시 프롬프트 설정:")
    print(f"   - 프롬프트 사용: {prompt_config.get('use_prompt', False)}")
    print(f"   - 프롬프트 스타일: {prompt_config.get('prompt_style', 'none')}")
    if prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']:
        print(f"   - TF-IDF 키워드 수: {prompt_config.get('tfidf_top_k', 5)}개")
    
    print(f"\n🗣️ 추론 시 구어체 전처리 설정:")
    print(f"   - 슬랭 정규화: {preprocess_config.get('normalize_slang', False)}")
    
    print(f"\n🔍 6. 정규표현식 패턴 활용:")
    print(f"   - 6-1. 화자 수 명시: {regex_pattern_config.get('use_speaker_count', True)}")
    print(f"   - 6-2. PII 마스킹 유지 지시: {regex_pattern_config.get('use_pii_instruction', True)}")
    
    print(f"\n⚙️ 동적 추론 설정:")
    print(f"   - 동적 설정 사용: {use_dynamic}")
    if use_dynamic:
        print(f"   - 길이 기반: {dynamic_inference_config.get('use_length_based', False)}")
        print(f"   - 주제 기반: {dynamic_inference_config.get('use_topic_based', False)}")
    
    # 5-1. TF-IDF extractor 사용 여부 확인
    use_tfidf = prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']
    tfidf_ext = tfidf_extractor if use_tfidf else None
    
    if use_tfidf:
        print(f"\n📊 TF-IDF 키워드 추출 활성화")
    
    preprocessor = Preprocess(
        config['tokenizer']['bos_token'], 
        config['tokenizer']['eos_token'],
        prompt_config=prompt_config,               # 3-2. 프롬프트 설정 전달
        preprocess_config=preprocess_config,       # 4. 구어체 전처리 설정 전달
        tfidf_extractor=tfidf_ext,                 # 5-1. TF-IDF extractor 전달
        regex_pattern_config=regex_pattern_config  # 6. 정규표현식 패턴 설정 전달
    )

    data_path = config['general']['data_path']
    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
    
    # 5-2. 동적 추론을 위해 원본 텍스트 길이 정보 필요
    dialogue_lengths = []
    if use_dynamic:
        # test_data에서 대화문 길이 계산
        for idx in range(len(test_data)):
            dialogue = test_data.iloc[idx]['dialogue']
            dialogue_lengths.append(len(dialogue))
        print(f"\n📏 대화문 길이 분포: min={min(dialogue_lengths)}, max={max(dialogue_lengths)}, avg={sum(dialogue_lengths)/len(dialogue_lengths):.0f}")
    
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    
    # 5-2. 동적 추론 시 배치 단위가 아닌 개별 샘플 처리 필요
    if use_dynamic:
        print("\n🚀 동적 추론 모드로 생성 시작...")
        sample_idx = 0
        use_topic = dynamic_inference_config.get('use_topic_based', True)
        use_length = dynamic_inference_config.get('use_length_based', True)
        
        with torch.no_grad():
            for item in tqdm(dataloader):
                batch_size = len(item['ID'])
                text_ids.extend(item['ID'])
                
                for i in range(batch_size):
                    # 개별 샘플에 대한 동적 설정 계산 (원본 대화문 사용)
                    dialogue_text = test_data.iloc[sample_idx]['dialogue']
                    dynamic_cfg = get_dynamic_inference_config(dialogue_text, use_topic=use_topic, use_length=use_length)
                    
                    # 단일 샘플 생성
                    single_input_ids = item['input_ids'][i:i+1].to(device)
                    generated_ids = generate_model.generate(
                        input_ids=single_input_ids,
                        no_repeat_ngram_size=dynamic_cfg['no_repeat_ngram_size'],
                        early_stopping=config['inference']['early_stopping'],
                        max_length=dynamic_cfg['max_new_tokens'],
                        num_beams=dynamic_cfg['num_beams'],
                        length_penalty=dynamic_cfg['length_penalty'],
                    )
                    
                    # 3-1 개선: 화자 태그 보존하면서 디코딩
                    result = decode_with_speaker_tags(tokenizer, generated_ids[0])
                    summary.append(result)
                    sample_idx += 1
    else:
        # 기존 배치 처리 방식
        with torch.no_grad():
            for item in tqdm(dataloader):
                text_ids.extend(item['ID'])
                generated_ids = generate_model.generate(
                    input_ids=item['input_ids'].to(device),
                    no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                    early_stopping=config['inference']['early_stopping'],
                    max_length=config['inference']['generate_max_length'],
                    num_beams=config['inference']['num_beams'],
                )
                for ids in generated_ids:
                    # 3-1 개선: 화자 태그 보존하면서 디코딩
                    result = decode_with_speaker_tags(tokenizer, ids)
                    summary.append(result)

    # 후처리 적용 (띄어쓰기 교정)
    preprocessed_summary = [postprocess_summary(s) for s in summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary": preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [30]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------
---------- Checkpoint : ./prediction_kobart_v1/checkpoint-4674 ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



📌 등록된 Special Tokens (22개)
   새로 추가된 토큰: 22개
   토크나이저 vocab size: 30022
   모델 임베딩 사이즈: 30022
---------- Load tokenizer & model complete ----------

📋 추론 시 프롬프트 설정:
   - 프롬프트 사용: True
   - 프롬프트 스타일: balanced

🗣️ 추론 시 구어체 전처리 설정:
   - 슬랭 정규화: True

🔍 6. 정규표현식 패턴 활용:
   - 6-1. 화자 수 명시: True
   - 6-2. PII 마스킹 유지 지시: True

⚙️ 동적 추론 설정:
   - 동적 설정 사용: True
   - 길이 기반: True
   - 주제 기반: True
------------------------------------------------------------------------------------------------------------------------------------------------------
[원본] test_data dialogue:
#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. 
#Person2#: 네, 말씀하세요... 
#Person1#: 이걸 오늘 오후까지 모든 직원들에게 사내 메모로 보내야 해요. 준비됐나요? 
#Person2#: 네, 말씀하세요. 
#Person1#: 모든 직원에게 알립니다... 즉시 발효되어 모든 사내 통신은 이메일과 공식 메모로만 제...
------------------------------------------------------------------------------------------------------------------------------------------------------
[프롬프트 적용 후] encoder_input_test[0]:
다음은 2명이 나눈 대화입니다. 두 사람의 입장을 균형 있게 반영해서 요약해 주세요

100%|██████████| 16/16 [01:24<00:00,  5.29s/it]



In [31]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,#Person1# 은 Ms. Dawson에게 이메일과 공식 메모로만 제한되는 사내 ...
1,test_1,#Person1# 과 #Person2# 는 교통체증으로 인해 출퇴근에 어려움을 겪고...
2,test_2,#Person1# 은 Kate에게 Masha와 Hero가 두 달 동안 별거하다가 결...
3,test_3,#Person1# 은 Brian에게 생일 파티에서 함께 춤을 추자고 초대합니다.
4,test_4,#Person1# 과 #Person2# 는 올림픽 공원의 크기와 시설에 대해 이야기...
...,...,...
494,test_495,"Jack은 Charlie에게 학교 후 집에서 그녀와 비디오 게임을 하자고 제안하고,..."
495,test_496,#Person2# 는 시골 음악에 관심을 가지게 된 계기와 라디오 방송국에서 일하게...
496,test_497,Alice는 #Person1# 에게 세탁기에 비누가 들어 있지 않아 따로 넣어야 한...
497,test_498,"Matthew는 임대 계약을 갱신하고 싶지 않아 집을 찾고 있으며, Mrs. Tho..."


In [ ]:
# ============================================================================
# 🚀 리더보드 점수 향상 종합 전략
# ============================================================================
# 현재 모델: digit82/kobart-summarization (리더보드 48.32점 달성 가능)
# 목표: 체계적인 단계별 최적화로 점수 극대화
# ============================================================================


╔══════════════════════════════════════════════════════════════════════════════╗
║                    🎯 리더보드 점수 향상 4단계 전략                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║ Phase 1: Quick Wins (즉시 적용)                     예상 향상: +0.5~1.5점    ║
║   └─ 형태소 기반 ROUGE 평가, 띄어쓰기 교정, N-Best Reranking                 ║
║                                                                              ║
║ Phase 2: Hyperparameter Optimization               예상 향상: +0.3~0.8점    ║
║   └─ Grid Search (num_beams, length_penalty, repetition_penalty)            ║
║                                                                              ║
║ Phase 3: Model Ensemble                            예상 향상: +0.5~1.0점    ║
║   └─ digit82 + gogamza 모델 앙상블                                          ║
║                                                                            ║
║ Phase 4: Advanced Techniques                       예상 향상: +0.5~1.5점    ║
║   └─ Knowledge Distillation, Back-translation Augmentation                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║ 예상 최종 점수: 49.5~51.0점                                                  ║
╚══════════════════════════════════════════════════════════════════════════════╝



╔══════════════════════════════════════════════════════════════════════════════╗
║                    🎯 리더보드 점수 향상 4단계 전략                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║ Phase 1: Quick Wins (즉시 적용)                     예상 향상: +0.5~1.5점    ║
║   └─ 형태소 기반 ROUGE 평가, 띄어쓰기 교정, N-Best Reranking                 ║
║                                                                              ║
║ Phase 2: Hyperparameter Optimization               예상 향상: +0.3~0.8점    ║
║   └─ Grid Search (num_beams, length_penalty, repetition_penalty)            ║
║                                                                              ║
║ Phase 3: Model Ensemble                            예상 향상: +0.5~1.0점    ║
║   └─ digit82 + gogamza 모델 앙상블                                          ║
║                                                                              ║# ============================================================================
# 

In [33]:
# ============================================================================
# 📦 Phase 1: Quick Wins - 필수 라이브러리 및 유틸리티
# ============================================================================

import numpy as np
import random
from kiwipiepy import Kiwi
from rouge import Rouge

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()
rouge_scorer = Rouge()

# ⚠️ 중요: 제거할 토큰 (화자 태그는 제외!)
# 화자 태그 (#Person1#, #Person2# 등)는 보존해야 함
REMOVE_TOKENS = ['<usr>', '<s>', '</s>', '<pad>']

# Seed 고정 함수
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

# ⚠️ 수정된 디코딩 함수: 화자 태그 보존!
def decode_with_speaker_tags(tokenizer, output_ids, remove_tokens=None):
    """
    화자 태그(#Person1#, #Person2# 등)를 보존하면서 디코딩
    """
    if remove_tokens is None:
        remove_tokens = REMOVE_TOKENS
    
    text = tokenizer.decode(output_ids, skip_special_tokens=False)
    for token in remove_tokens:
        text = text.replace(token, " ")
    text = ' '.join(text.split())
    return text.strip()

# 형태소 토큰화 함수
def morpheme_tokenize(text):
    """Kiwi 형태소 분석"""
    if not text or not text.strip():
        return ""
    tokens = kiwi.tokenize(text.strip())
    return " ".join([t.form for t in tokens])

# 형태소 기반 ROUGE 계산
def compute_morpheme_rouge(pred, ref):
    """형태소 기반 ROUGE-L F1 계산"""
    pred_morph = morpheme_tokenize(pred)
    ref_morph = morpheme_tokenize(ref)
    
    if not pred_morph or not ref_morph:
        return {'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}
    
    try:
        scores = rouge_scorer.get_scores(pred_morph, ref_morph)
        return {
            'rouge-1': scores[0]['rouge-1']['f'],
            'rouge-2': scores[0]['rouge-2']['f'],
            'rouge-l': scores[0]['rouge-l']['f']
        }
    except:
        return {'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}

# 띄어쓰기 교정 함수
def postprocess_spacing(text):
    """Kiwi를 이용한 띄어쓰기 교정"""
    if not text or not text.strip():
        return text
    return kiwi.space(text, reset_whitespace=False)

# 후처리 함수 (화자 태그 보존)
def postprocess_summary(text):
    """요약문 후처리 (화자 태그 보존)"""
    text = text.strip()
    text = postprocess_spacing(text)
    text = ' '.join(text.split())
    text = text.replace(' .', '.').replace(' ,', ',').replace(' ?', '?').replace(' !', '!')
    return text

# ==============================================================================
# 🔧 길이 분석 기반 트렁케이션 전략
# ==============================================================================

# ✨ 전략 1: 뒤에서부터 자르기 (후반부 결론 보존)
def truncate_dialogue_tail(text, max_char=1500):
    """긴 대화는 뒷부분(결론)을 우선 보존하여 자름"""
    if len(text) <= max_char:
        return text
    return text[-max_char:]

# ✨ 전략 2: 앞30% + 뒤70% 보존 (도입부 + 결론 모두 살림)
def truncate_dialogue_hybrid(text, max_char=1500, front_ratio=0.3):
    """앞부분 30% + 뒷부분 70% 보존 (도입 + 결론 둘 다 살림)"""
    if len(text) <= max_char:
        return text
    front_chars = int(max_char * front_ratio)
    back_chars = max_char - front_chars
    return text[:front_chars] + " ... " + text[-back_chars:]

# ✨ 전략 3: 토큰 기반 트렁케이션 (정확한 토큰 수 제어)
def truncate_dialogue_tokens(text, tokenizer, max_tokens, strategy='tail'):
    """
    토큰 기반으로 트렁케이션 (문자 기반보다 정확)
    strategy: 'tail' (뒷부분 보존), 'head' (앞부분 보존), 'hybrid' (앞30%+뒤70%)
    """
    if not text:
        return text
    
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    
    if len(token_ids) <= max_tokens:
        return text
    
    if strategy == 'tail':
        # 뒷부분 보존 (결론 중심)
        truncated_ids = token_ids[-max_tokens:]
    elif strategy == 'head':
        # 앞부분 보존
        truncated_ids = token_ids[:max_tokens]
    elif strategy == 'hybrid':
        # 앞 30% + 뒤 70%
        front_tokens = int(max_tokens * 0.3)
        back_tokens = max_tokens - front_tokens
        truncated_ids = token_ids[:front_tokens] + token_ids[-back_tokens:]
    else:
        truncated_ids = token_ids[-max_tokens:]
    
    try:
        return tokenizer.decode(truncated_ids, skip_special_tokens=True)
    except:
        return text[-max_tokens*3:]  # fallback: 문자 기반

# ✨ 전략 4: 대화 길이에 따른 동적 max_length 설정
def get_dynamic_max_length(text_length, thresholds=None):
    """
    대화 길이에 따라 동적으로 encoder_max_len 결정
    짧은 대화: 256 토큰, 긴 대화: 512 토큰
    """
    if thresholds is None:
        thresholds = {
            500: 256,    # 500자 이하 → 256 토큰
            1000: 384,   # 1000자 이하 → 384 토큰
            2000: 512,   # 2000자 이하 → 512 토큰
            float('inf'): 768  # 그 이상 → 768 토큰
        }
    
    for char_limit, token_limit in sorted(thresholds.items()):
        if text_length <= char_limit:
            return token_limit
    return 512  # default

# ✨ 프롬프트 추가 함수
def apply_summary_prompt(dialogue, prompt=None):
    """요약 유도 프롬프트를 대화 뒤에 추가"""
    if prompt is None:
        return dialogue
    return f"{dialogue} {prompt}"

# ==============================================================================
# 📊 EDA 기반 최적 설정 (실험 결과 반영)
# ==============================================================================

# 권장 설정 (EDA 기반)
LENGTH_CONFIG = {
    # dialogue 처리
    'truncation_strategy': 'tail',  # 'tail', 'head', 'hybrid' 중 선택
    'max_dialogue_chars': 1500,     # 문자 기반 트렁케이션 기준
    'max_dialogue_tokens': 512,     # 토큰 기반 트렁케이션 기준
    'use_dynamic_length': False,    # 동적 길이 사용 여부
    
    # summary 처리
    'decoder_max_len': 100,         # 요약 최대 토큰 (95% 커버)
    
    # 프롬프트
    'use_prompt': False,
    'prompt_text': '요약:'
}

print("✅ Phase 1 유틸리티 로드 완료")
print(f"   - Kiwi 형태소 분석기 초기화")
print(f"   - 형태소 기반 ROUGE 함수")
print(f"   - 띄어쓰기 교정 함수")
print(f"   - ⚠️ 화자 태그 보존 디코딩 함수")
print(f"   - 🔧 트렁케이션 전략 함수들:")
print(f"       • truncate_dialogue_tail (뒷부분 보존)")
print(f"       • truncate_dialogue_hybrid (앞30%+뒤70%)")
print(f"       • truncate_dialogue_tokens (토큰 기반)")
print(f"       • get_dynamic_max_length (동적 길이)")
print(f"   - 제거 토큰: {REMOVE_TOKENS}")
print(f"   - 보존 토큰: #Person1#, #Person2#, ... (화자 태그)")

✅ Phase 1 유틸리티 로드 완료
   - Kiwi 형태소 분석기 초기화
   - 형태소 기반 ROUGE 함수
   - 띄어쓰기 교정 함수
   - ⚠️ 화자 태그 보존 디코딩 함수
   - 🔧 트렁케이션 전략 함수들:
       • truncate_dialogue_tail (뒷부분 보존)
       • truncate_dialogue_hybrid (앞30%+뒤70%)
       • truncate_dialogue_tokens (토큰 기반)
       • get_dynamic_max_length (동적 길이)
   - 제거 토큰: ['<usr>', '<s>', '</s>', '<pad>']
   - 보존 토큰: #Person1#, #Person2#, ... (화자 태그)


In [34]:
# ============================================================================
# 🎯 Phase 1-A: Dev 데이터로 현재 성능 평가 (형태소 ROUGE)
# ============================================================================

print("=" * 80)
print("🎯 Phase 1-A: 현재 모델 성능 평가 (형태소 ROUGE)")
print("=" * 80)

# 디바이스 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# 모델 로드 (checkpoint 경로 설정 필요)
model_path = loaded_config['inference']['ckt_path']
print(f"📥 모델 로드: {model_path}")

model_name = loaded_config['general']['model_name']
eval_tokenizer = AutoTokenizer.from_pretrained(model_name)
eval_tokenizer.add_special_tokens({
    'additional_special_tokens': loaded_config['tokenizer']['special_tokens']
})

eval_model = BartForConditionalGeneration.from_pretrained(model_path)
eval_model.resize_token_embeddings(len(eval_tokenizer))
eval_model = eval_model.to(device)
eval_model.eval()

print("✅ 모델 로드 완료")

# Dev 데이터 로드
dev_df = pd.read_csv("./data/dev.csv")
print(f"📊 Dev 데이터: {len(dev_df)}개")

# 기본 추론 설정
BASE_GEN_CONFIG = {
    "num_beams": loaded_config['inference']['num_beams'],
    "no_repeat_ngram_size": loaded_config['inference']['no_repeat_ngram_size'],
    "max_length": loaded_config['inference']['generate_max_length'],
    "early_stopping": loaded_config['inference']['early_stopping'],
}

def evaluate_on_dev(model, tokenizer, df, gen_config, apply_postprocess=False):
    """Dev 데이터로 모델 평가 (화자 태그 보존)"""
    predictions = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="평가"):
        inputs = tokenizer(
            row['dialogue'],
            return_tensors="pt",
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            padding=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                **gen_config
            )
        
        # ⚠️ 수정: 화자 태그 보존하면서 디코딩
        pred = decode_with_speaker_tags(tokenizer, outputs[0])
        
        # 후처리 적용 (띄어쓰기 교정 등)
        if apply_postprocess:
            pred = postprocess_summary(pred)
        
        predictions.append(pred)
    
    # 형태소 ROUGE 계산
    rouge1_scores, rouge2_scores, rougel_scores = [], [], []
    
    for pred, ref in zip(predictions, df['summary'].tolist()):
        scores = compute_morpheme_rouge(pred, ref)
        rouge1_scores.append(scores['rouge-1'])
        rouge2_scores.append(scores['rouge-2'])
        rougel_scores.append(scores['rouge-l'])
    
    return {
        'predictions': predictions,
        'rouge-1': np.mean(rouge1_scores) * 100,
        'rouge-2': np.mean(rouge2_scores) * 100,
        'rouge-l': np.mean(rougel_scores) * 100,
        'avg_len': np.mean([len(p) for p in predictions])
    }

# 기본 성능 평가 (후처리 없이)
print("\n🔬 기본 성능 평가 (후처리 없이)...")
base_results = evaluate_on_dev(eval_model, eval_tokenizer, dev_df, BASE_GEN_CONFIG, apply_postprocess=False)

print(f"\n📊 기본 성능 (형태소 ROUGE):")
print(f"   ROUGE-1: {base_results['rouge-1']:.2f}")
print(f"   ROUGE-2: {base_results['rouge-2']:.2f}")
print(f"   ROUGE-L: {base_results['rouge-l']:.2f}")
print(f"   평균 길이: {base_results['avg_len']:.1f}자")

# 예상 리더보드 점수 (형태소 ROUGE-L × 1.32)
estimated_leaderboard = base_results['rouge-l'] * 1.32
print(f"\n🎯 예상 리더보드 점수: {estimated_leaderboard:.2f}")

# 샘플 출력 (화자 태그 확인)
print("\n📋 예측 샘플 (화자 태그 확인):")
for i in range(3):
    print(f"\n[Sample {i}]")
    print(f"  Pred: {base_results['predictions'][i][:100]}...")
    print(f"  Gold: {dev_df['summary'].iloc[i][:100]}...")

🎯 Phase 1-A: 현재 모델 성능 평가 (형태소 ROUGE)
Device: cuda:0
📥 모델 로드: ./prediction_kobart_v1/checkpoint-4674


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ 모델 로드 완료
📊 Dev 데이터: 499개

🔬 기본 성능 평가 (후처리 없이)...


평가: 100%|██████████| 499/499 [01:21<00:00,  6.16it/s]




📊 기본 성능 (형태소 ROUGE):
   ROUGE-1: 41.05
   ROUGE-2: 16.18
   ROUGE-L: 35.61
   평균 길이: 73.3자

🎯 예상 리더보드 점수: 47.00

📋 예측 샘플 (화자 태그 확인):

[Sample 0]
  Pred: #Person1# 은 최근에 감기에 걸렸지만 천식 검사를 위해 폐 전문의에게 가보시길 권장합니다....
  Gold: #Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다....

[Sample 1]
  Pred: #Person1# 과 야 Jimmy는 3시 30분에 체육관에서 운동하기로 했습니다....
  Gold: #Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다....

[Sample 2]
  Pred: #Person1# 과 건강에 안 좋은 음식을 끊고 건강을 유지하려고 합니다....
  Gold: #Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#는 자신의 건강한 식단을 #Person1#에게 공유합니다....


In [35]:
# ============================================================================
# 🎯 Phase 1-B: 후처리 적용 효과 측정
# ============================================================================

print("=" * 80)
print("🎯 Phase 1-B: 띄어쓰기 교정 후처리 효과 측정")
print("=" * 80)

# 후처리 적용 평가
print("\n🔬 후처리 적용 성능 평가...")
postproc_results = evaluate_on_dev(eval_model, eval_tokenizer, dev_df, BASE_GEN_CONFIG, apply_postprocess=True)

print(f"\n📊 후처리 적용 후 성능 (형태소 ROUGE):")
print(f"   ROUGE-1: {postproc_results['rouge-1']:.2f} ({postproc_results['rouge-1'] - base_results['rouge-1']:+.2f})")
print(f"   ROUGE-2: {postproc_results['rouge-2']:.2f} ({postproc_results['rouge-2'] - base_results['rouge-2']:+.2f})")
print(f"   ROUGE-L: {postproc_results['rouge-l']:.2f} ({postproc_results['rouge-l'] - base_results['rouge-l']:+.2f})")
print(f"   평균 길이: {postproc_results['avg_len']:.1f}자")

# 예상 리더보드 점수
estimated_leaderboard_pp = postproc_results['rouge-l'] * 1.32
print(f"\n🎯 예상 리더보드 점수: {estimated_leaderboard_pp:.2f} ({estimated_leaderboard_pp - estimated_leaderboard:+.2f})")

🎯 Phase 1-B: 띄어쓰기 교정 후처리 효과 측정

🔬 후처리 적용 성능 평가...


평가: 100%|██████████| 499/499 [01:21<00:00,  6.15it/s]




📊 후처리 적용 후 성능 (형태소 ROUGE):
   ROUGE-1: 41.12 (+0.07)
   ROUGE-2: 16.27 (+0.09)
   ROUGE-L: 35.67 (+0.06)
   평균 길이: 73.6자

🎯 예상 리더보드 점수: 47.08 (+0.08)


In [36]:
# ============================================================================
# 🔬 Phase 1-C: 트렁케이션 전략 비교 실험
# ============================================================================

print("=" * 80)
print("🔬 Phase 1-C: 트렁케이션 전략 비교 실험")
print("=" * 80)

# 1. 먼저 데이터 길이 분포 확인
train_df_for_eda = pd.read_csv("./data/train.csv")
dev_df_for_eda = pd.read_csv("./data/dev.csv")

print("\n📊 Train 데이터 길이 분포:")
train_dialog_len = train_df_for_eda['dialogue'].apply(len)
train_summary_len = train_df_for_eda['summary'].apply(len)
print(f"  Dialogue: mean={train_dialog_len.mean():.0f}, 75%={train_dialog_len.quantile(0.75):.0f}, max={train_dialog_len.max()}")
print(f"  Summary:  mean={train_summary_len.mean():.0f}, 75%={train_summary_len.quantile(0.75):.0f}, 95%={train_summary_len.quantile(0.95):.0f}")

# 토큰 수 분포 (샘플링)
sample_dialogs = train_df_for_eda['dialogue'].sample(min(200, len(train_df_for_eda)), random_state=42)
token_counts = [len(eval_tokenizer.encode(d, add_special_tokens=False)) for d in sample_dialogs]
print(f"\n📊 Dialogue 토큰 수 분포 (샘플 200개):")
print(f"  mean={np.mean(token_counts):.0f}, 75%={np.percentile(token_counts, 75):.0f}, max={np.max(token_counts)}")

# 2. 트렁케이션 전략별 Dev 성능 비교
print("\n" + "=" * 80)
print("🧪 트렁케이션 전략별 Dev ROUGE 비교 (50개 샘플)")
print("=" * 80)

TRUNCATION_EXPERIMENTS = [
    {'name': 'no_trunc', 'strategy': None, 'max_tokens': 1024},
    {'name': 'tail_512', 'strategy': 'tail', 'max_tokens': 512},
    {'name': 'tail_384', 'strategy': 'tail', 'max_tokens': 384},
    {'name': 'hybrid_512', 'strategy': 'hybrid', 'max_tokens': 512},
    {'name': 'head_512', 'strategy': 'head', 'max_tokens': 512},
]

def evaluate_truncation_strategy(model, tokenizer, df, strategy, max_tokens, sample_size=50):
    """트렁케이션 전략별 평가"""
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    predictions = []
    
    for idx, row in sample_df.iterrows():
        dialogue = row['dialogue']
        
        # 트렁케이션 적용
        if strategy is not None:
            dialogue = truncate_dialogue_tokens(dialogue, tokenizer, max_tokens, strategy=strategy)
        
        inputs = tokenizer(
            dialogue,
            return_tensors="pt",
            max_length=max_tokens,
            truncation=True,
            padding=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                **BASE_GEN_CONFIG
            )
        
        pred = decode_with_speaker_tags(tokenizer, outputs[0])
        pred = postprocess_summary(pred)
        predictions.append(pred)
    
    # ROUGE 계산
    rougel_scores = []
    for pred, ref in zip(predictions, sample_df['summary'].tolist()):
        scores = compute_morpheme_rouge(pred, ref)
        rougel_scores.append(scores['rouge-l'])
    
    return {
        'rouge-l': np.mean(rougel_scores) * 100,
        'predictions': predictions
    }

# 실험 실행
trunc_results = []
for exp in TRUNCATION_EXPERIMENTS:
    print(f"\n🔧 Testing: {exp['name']} (strategy={exp['strategy']}, max_tokens={exp['max_tokens']})")
    result = evaluate_truncation_strategy(
        eval_model, eval_tokenizer, dev_df,
        strategy=exp['strategy'],
        max_tokens=exp['max_tokens'],
        sample_size=50
    )
    trunc_results.append({
        'name': exp['name'],
        'strategy': exp['strategy'],
        'max_tokens': exp['max_tokens'],
        'rouge-l': result['rouge-l'],
        'estimated_lb': result['rouge-l'] * 1.32
    })
    print(f"   ROUGE-L: {result['rouge-l']:.2f}, 예상 LB: {result['rouge-l'] * 1.32:.2f}")

# 결과 정렬
trunc_results_sorted = sorted(trunc_results, key=lambda x: x['rouge-l'], reverse=True)

print("\n" + "=" * 80)
print("📊 트렁케이션 전략 비교 결과 (ROUGE-L 순)")
print("=" * 80)
for i, r in enumerate(trunc_results_sorted):
    marker = "🏆" if i == 0 else "  "
    print(f"{marker} {i+1}. {r['name']:15} | ROUGE-L: {r['rouge-l']:.2f} | 예상 LB: {r['estimated_lb']:.2f}")

BEST_TRUNCATION = trunc_results_sorted[0]
print(f"\n✅ 최적 전략: {BEST_TRUNCATION['name']} (strategy={BEST_TRUNCATION['strategy']}, max_tokens={BEST_TRUNCATION['max_tokens']})")

🔬 Phase 1-C: 트렁케이션 전략 비교 실험

📊 Train 데이터 길이 분포:
  Dialogue: mean=406, 75%=500, max=2165
  Summary:  mean=86, 75%=104, 95%=149

📊 Dialogue 토큰 수 분포 (샘플 200개):
  mean=157, 75%=190, max=777

🧪 트렁케이션 전략별 Dev ROUGE 비교 (50개 샘플)

🔧 Testing: no_trunc (strategy=None, max_tokens=1024)

📊 Dialogue 토큰 수 분포 (샘플 200개):
  mean=157, 75%=190, max=777

🧪 트렁케이션 전략별 Dev ROUGE 비교 (50개 샘플)

🔧 Testing: no_trunc (strategy=None, max_tokens=1024)
   ROUGE-L: 33.72, 예상 LB: 44.51

🔧 Testing: tail_512 (strategy=tail, max_tokens=512)
   ROUGE-L: 33.72, 예상 LB: 44.51

🔧 Testing: tail_512 (strategy=tail, max_tokens=512)
   ROUGE-L: 33.72, 예상 LB: 44.51

🔧 Testing: tail_384 (strategy=tail, max_tokens=384)
   ROUGE-L: 33.72, 예상 LB: 44.51

🔧 Testing: tail_384 (strategy=tail, max_tokens=384)
   ROUGE-L: 33.72, 예상 LB: 44.51

🔧 Testing: hybrid_512 (strategy=hybrid, max_tokens=512)
   ROUGE-L: 33.72, 예상 LB: 44.51

🔧 Testing: hybrid_512 (strategy=hybrid, max_tokens=512)
   ROUGE-L: 33.72, 예상 LB: 44.51

🔧 Testing: head_512 (stra

In [39]:
# ============================================================================
# 📤 Phase 1-D: 최적 트렁케이션 전략으로 제출 파일 생성
# ============================================================================

print("=" * 80)
print("📤 Phase 1-D: 최적 트렁케이션 전략 적용 제출 파일 생성")
print("=" * 80)

# Test 데이터 로드 (커널 재시작 후에도 동작하도록)
test_df = pd.read_csv(os.path.join(data_path, 'test.csv'))
print(f"📊 Test 데이터: {len(test_df)}개")

def generate_submission_with_truncation(model, tokenizer, test_df, gen_config, 
                                        truncation_strategy, max_tokens, output_path):
    """트렁케이션 전략을 적용한 제출 파일 생성"""
    summaries = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="추론"):
        dialogue = row['dialogue']
        
        # 트렁케이션 적용
        if truncation_strategy is not None:
            dialogue = truncate_dialogue_tokens(dialogue, tokenizer, max_tokens, strategy=truncation_strategy)
        
        inputs = tokenizer(
            dialogue,
            return_tensors="pt",
            max_length=max_tokens,
            truncation=True,
            padding=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                **gen_config
            )
        
        pred = decode_with_speaker_tags(tokenizer, outputs[0])
        pred = postprocess_summary(pred)
        summaries.append(pred)
    
    # 제출 파일 생성
    submission = pd.DataFrame({
        'fname': test_df['fname'],
        'summary': summaries
    })
    
    submission.to_csv(output_path, index=False)
    
    # 통계 출력
    has_speaker = sum(1 for s in summaries if '#Person' in s)
    print(f"✅ 저장 완료: {output_path}")
    print(f"   평균 길이: {np.mean([len(s) for s in summaries]):.1f}자")
    print(f"   화자 태그 포함: {has_speaker}/{len(summaries)}개 ({has_speaker/len(summaries)*100:.1f}%)")
    
    return submission

# 최적 전략 사용 (위 실험 결과 기반)
# 실험 결과가 없으면 기본값 사용
try:
    best_strategy = BEST_TRUNCATION['strategy']
    best_max_tokens = BEST_TRUNCATION['max_tokens']
    print(f"📌 적용 전략: {BEST_TRUNCATION['name']}")
except:
    best_strategy = 'tail'
    best_max_tokens = 512
    print(f"📌 기본 전략 사용: tail, 512 tokens")

# 제출 파일 생성
print(f"\n🚀 제출 파일 생성 중... (strategy={best_strategy}, max_tokens={best_max_tokens})")

submission_trunc = generate_submission_with_truncation(
    eval_model, eval_tokenizer, test_df, 
    BASE_GEN_CONFIG,
    truncation_strategy=best_strategy,
    max_tokens=best_max_tokens,
    output_path="./prediction/submit_phase1d_truncation.csv"
)

# 샘플 확인
print("\n📋 제출 샘플:")
for i in range(3):
    print(f"\n[Test {i}]")
    summary = submission_trunc['summary'].iloc[i]
    has_tag = "✅" if "#Person" in summary else "❌"
    print(f"  {has_tag} {summary[:100]}...")

📤 Phase 1-D: 최적 트렁케이션 전략 적용 제출 파일 생성
📊 Test 데이터: 499개
📌 적용 전략: no_trunc

🚀 제출 파일 생성 중... (strategy=None, max_tokens=1024)


추론:   0%|          | 0/499 [00:00<?, ?it/s]

추론: 100%|██████████| 499/499 [01:23<00:00,  5.99it/s]

✅ 저장 완료: ./prediction/submit_phase1d_truncation.csv
   평균 길이: 73.4자
   화자 태그 포함: 360/499개 (72.1%)

📋 제출 샘플:

[Test 0]
  ❌ Ms. Dawson은 사내 메모를 작성하고 배포하기 위해 부서장에게 요청합니다....

[Test 1]
  ✅ #Person1# 과 교통 체증으로 인해 대중교통으로 출퇴근하는 것이 더 나은 선택이 될 수 있다고 생각합니다. 또한, 출근할 때 차를 타지 않기로 했습니다....

[Test 2]
  ❌ Kate는 Masha와 Hero가 두 달 동안 별거하다가 결국 이혼을 하게 되었다고 전하며, 그들은 양육권을 가진다....


In [40]:
# ============================================================================
# 🎯 Phase 2-A: Grid Search for 하이퍼파라미터 최적화
# ============================================================================

print("=" * 80)
print("🎯 Phase 2-A: Grid Search 하이퍼파라미터 최적화")
print("=" * 80)

# Grid Search 설정
GRID_SEARCH_CONFIGS = [
    {"name": "baseline", "num_beams": 4, "length_penalty": 1.0, "repetition_penalty": 1.0},
    {"name": "nb6", "num_beams": 6, "length_penalty": 1.0, "repetition_penalty": 1.0},
    {"name": "nb8", "num_beams": 8, "length_penalty": 1.0, "repetition_penalty": 1.0},
    {"name": "lp0.9", "num_beams": 6, "length_penalty": 0.9, "repetition_penalty": 1.0},
    {"name": "lp1.1", "num_beams": 6, "length_penalty": 1.1, "repetition_penalty": 1.0},
    {"name": "rp1.1", "num_beams": 6, "length_penalty": 1.0, "repetition_penalty": 1.1},
    {"name": "rp1.2", "num_beams": 6, "length_penalty": 1.0, "repetition_penalty": 1.2},
    {"name": "best_combo", "num_beams": 7, "length_penalty": 1.0, "repetition_penalty": 1.2},
]

def run_grid_search(model, tokenizer, df, configs, sample_size=100):
    """Grid Search 실행 (샘플 데이터로 빠른 탐색)"""
    results = []
    
    # 샘플링 (빠른 탐색)
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    
    for config in configs:
        print(f"\n🔧 Testing: {config['name']}")
        
        gen_config = {
            "num_beams": config["num_beams"],
            "length_penalty": config.get("length_penalty", 1.0),
            "repetition_penalty": config.get("repetition_penalty", 1.0),
            "no_repeat_ngram_size": 3,
            "max_length": 100,
            "early_stopping": True,
        }
        
        predictions = []
        for idx, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc=config['name']):
            inputs = tokenizer(
                row['dialogue'],
                return_tensors="pt",
                max_length=loaded_config['tokenizer']['encoder_max_len'],
                truncation=True,
                padding=True
            ).to(device)
            
            with torch.no_grad():
                outputs = model.generate(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    **gen_config
                )
            
            # ⚠️ 수정: 화자 태그 보존하면서 디코딩
            pred = decode_with_speaker_tags(tokenizer, outputs[0])
            pred = postprocess_summary(pred)
            predictions.append(pred)
        
        # 형태소 ROUGE 계산
        rougel_scores = []
        for pred, ref in zip(predictions, sample_df['summary'].tolist()):
            scores = compute_morpheme_rouge(pred, ref)
            rougel_scores.append(scores['rouge-l'])
        
        avg_rouge = np.mean(rougel_scores) * 100
        avg_len = np.mean([len(p) for p in predictions])
        
        results.append({
            'name': config['name'],
            'config': config,
            'rouge-l': avg_rouge,
            'avg_len': avg_len,
            'estimated_lb': avg_rouge * 1.32,
            'predictions': predictions  # 화자 태그 확인용
        })
        
        print(f"   ROUGE-L: {avg_rouge:.2f}, 길이: {avg_len:.1f}, 예상 LB: {avg_rouge * 1.32:.2f}")
    
    return results

# Grid Search 실행
print("\n🚀 Grid Search 실행 중 (100개 샘플)...")
grid_results = run_grid_search(eval_model, eval_tokenizer, dev_df, GRID_SEARCH_CONFIGS, sample_size=100)

# 결과 정렬 및 출력
grid_results_sorted = sorted(grid_results, key=lambda x: x['rouge-l'], reverse=True)

print("\n" + "=" * 80)
print("📊 Grid Search 결과 (ROUGE-L 순)")
print("=" * 80)
for i, r in enumerate(grid_results_sorted):
    print(f"{i+1}. {r['name']:15} | ROUGE-L: {r['rouge-l']:.2f} | 예상 LB: {r['estimated_lb']:.2f} | 길이: {r['avg_len']:.1f}")

best_config = grid_results_sorted[0]
print(f"\n🏆 최적 설정: {best_config['name']}")
print(f"   Config: {best_config['config']}")

# 화자 태그 확인
print("\n📋 화자 태그 확인 (최적 설정 샘플):")
for i in range(3):
    print(f"  [{i}] {best_config.get('predictions', ['N/A'])[i][:80]}...")

🎯 Phase 2-A: Grid Search 하이퍼파라미터 최적화

🚀 Grid Search 실행 중 (100개 샘플)...

🔧 Testing: baseline


baseline: 100%|██████████| 100/100 [00:16<00:00,  6.16it/s]



   ROUGE-L: 34.43, 길이: 73.4, 예상 LB: 45.44

🔧 Testing: nb6


nb6: 100%|██████████| 100/100 [00:16<00:00,  5.93it/s]



   ROUGE-L: 35.49, 길이: 72.5, 예상 LB: 46.84

🔧 Testing: nb8


...

   ROUGE-L: 35.25, 길이: 71.5, 예상 LB: 46.53

🔧 Testing: rp1.2


rp1.2: 100%|██████████| 100/100 [00:17<00:00,  5.86it/s]



   ROUGE-L: 35.53, 길이: 72.0, 예상 LB: 46.90

🔧 Testing: best_combo


best_combo: 100%|██████████| 100/100 [00:17<00:00,  5.60it/s]



   ROUGE-L: 35.25, 길이: 72.1, 예상 LB: 46.54

📊 Grid Search 결과 (ROUGE-L 순)
1. nb8             | ROUGE-L: 35.61 | 예상 LB: 47.00 | 길이: 74.8
2. lp0.9           | ROUGE-L: 35.57 | 예상 LB: 46.95 | 길이: 71.2
3. rp1.2           | ROUGE-L: 35.53 | 예상 LB: 46.90 | 길이: 72.0
4. nb6             | ROUGE-L: 35.49 | 예상 LB: 46.84 | 길이: 72.5
5. lp1.1           | ROUGE-L: 35.45 | 예상 LB: 46.80 | 길이: 74.4
6. best_combo      | ROUGE-L: 35.25 | 예상 LB: 46.54 | 길이: 72.1
7. rp1.1           | ROUGE-L: 35.25 | 예상 LB: 46.53 | 길이: 71.5
8. baseline        | ROUGE-L: 34.43 | 예상 LB: 45.44 | 길이: 73.4

🏆 최적 설정: nb8
   Config: {'name': 'nb8', 'num_beams': 8, 'length_penalty': 1.0, 'repetition_penalty': 1.0}

📋 화자 태그 확인 (최적 설정 샘플):
  [0] #Person1# 은 그 사람을 건드리면 똑같은 당할 것이라며 그 사람과 멀리 떨어져 있는 것을 권장합니다....
  [1] #Person1# 은 지난 몇 달 동안 계획되지 않은 셧다운이 많아 정비 팀이 예방 정비를 할 시간이 부족하다고 생각합니다. 정비 팀은 예방 정...
  [2] Dr. Cardano는 한밤중에 오른쪽 발에 심한 통증이 생겼습니다. Bobby는 양말을 다 벗고 양쪽 발을 비교하며, 혈액 검사를 위해 3 층...


In [41]:
# ============================================================================
# 📤 Phase 2-A 제출: Grid Search 최적 하이퍼파라미터 적용 제출 파일 생성
# ============================================================================

print("=" * 80)
print("📤 Phase 2-A: Grid Search 최적 하이퍼파라미터 적용 제출 파일 생성")
print("=" * 80)

def generate_submission_with_best_config(model, tokenizer, test_df, gen_config, output_path):
    """
    Grid Search에서 찾은 최적 하이퍼파라미터로 제출 파일 생성
    """
    summaries = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="추론"):
        inputs = tokenizer(
            row['dialogue'],
            return_tensors="pt",
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            padding=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                **gen_config
            )
        
        pred = decode_with_speaker_tags(tokenizer, outputs[0])
        pred = postprocess_summary(pred)
        summaries.append(pred)
    
    # 제출 파일 생성
    submission = pd.DataFrame({
        'fname': test_df['fname'],
        'summary': summaries
    })
    
    submission.to_csv(output_path, index=False)
    
    # 통계 출력
    has_speaker = sum(1 for s in summaries if '#Person' in s)
    avg_len = np.mean([len(s) for s in summaries])
    print(f"✅ 저장 완료: {output_path}")
    print(f"   평균 길이: {avg_len:.1f}자")
    print(f"   화자 태그 포함: {has_speaker}/{len(summaries)}개 ({has_speaker/len(summaries)*100:.1f}%)")
    
    return submission

# Grid Search 최적 설정 가져오기
try:
    # grid_results_sorted에서 최적 설정 가져오기
    BEST_GRID_CONFIG = {
        "num_beams": best_config['config']['num_beams'],
        "length_penalty": best_config['config'].get('length_penalty', 1.0),
        "repetition_penalty": best_config['config'].get('repetition_penalty', 1.0),
        "no_repeat_ngram_size": 3,
        "max_length": 100,
        "early_stopping": True,
    }
    print(f"📌 Grid Search 최적 설정: {best_config['name']}")
    print(f"   - num_beams: {BEST_GRID_CONFIG['num_beams']}")
    print(f"   - length_penalty: {BEST_GRID_CONFIG['length_penalty']}")
    print(f"   - repetition_penalty: {BEST_GRID_CONFIG['repetition_penalty']}")
    print(f"   - Dev ROUGE-L: {best_config['rouge-l']:.2f}")
    print(f"   - 예상 LB: {best_config['estimated_lb']:.2f}")
except:
    # 기본 설정 사용
    BEST_GRID_CONFIG = {
        "num_beams": 6,
        "length_penalty": 1.0,
        "repetition_penalty": 1.2,
        "no_repeat_ngram_size": 3,
        "max_length": 100,
        "early_stopping": True,
    }
    print("⚠️ Grid Search 결과 없음, 기본 설정 사용")
    print(f"   설정: {BEST_GRID_CONFIG}")

# 제출 파일 생성
print(f"\n🚀 Grid Search 최적 설정으로 제출 파일 생성 중...")

submission_grid = generate_submission_with_best_config(
    eval_model, eval_tokenizer, test_df,
    BEST_GRID_CONFIG,
    output_path="./prediction/submit_phase2a_gridsearch.csv"
)

# 샘플 확인
print("\n📋 제출 샘플 (Grid Search 최적화):")
for i in range(5):
    print(f"\n[Test {i}]")
    summary = submission_grid['summary'].iloc[i]
    has_tag = "✅" if "#Person" in summary else "❌"
    print(f"  {has_tag} {summary[:100]}...")

📤 Phase 2-A: Grid Search 최적 하이퍼파라미터 적용 제출 파일 생성
📌 Grid Search 최적 설정: nb8
   - num_beams: 8
   - length_penalty: 1.0
   - repetition_penalty: 1.0
   - Dev ROUGE-L: 35.61
   - 예상 LB: 47.00

🚀 Grid Search 최적 설정으로 제출 파일 생성 중...


추론: 100%|██████████| 499/499 [01:32<00:00,  5.37it/s]

✅ 저장 완료: ./prediction/submit_phase2a_gridsearch.csv
   평균 길이: 75.8자
   화자 태그 포함: 357/499개 (71.5%)

📋 제출 샘플 (Grid Search 최적화):

[Test 0]
  ❌ Ms. Dawson은 사내 메모를 작성하고 배포하기 위해 부서장에게 요청합니다....

[Test 1]
  ✅ #Person1# 과 교통 체증으로 인해 대중교통으로 출퇴근하는 것이 더 나은 선택이 될 수 있다고 생각합니다. 또한, 출근할 때 차를 타지 않기로 했습니다....

[Test 2]
  ❌ Kate는 Masha와 Hero가 두 달 동안 별거하다가 결국 이혼을 하게 되었다고 Kate에게 설명한다....

[Test 3]
  ✅ #Person1# 은 Brian의 생일을 축하하기 위해 선물을 준비했습니다....

[Test 4]
  ✅ #Person1# 과 올림픽 스타디움의 전체 스타디움은 6월에 완공될 예정이며, 총 5000석이 있다....


In [42]:
# ============================================================================
# 🎯 Phase 2-B: N-Best Reranking 구현
# ============================================================================

print("=" * 80)
print("🎯 Phase 2-B: N-Best Reranking")
print("=" * 80)

def generate_nbest_candidates(model, tokenizer, input_text, config, n_candidates=8):
    """N-Best 후보 생성 (화자 태그 보존)"""
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=loaded_config['tokenizer']['encoder_max_len'],
        truncation=True,
        padding=True
    ).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            num_beams=config.get("num_beams", 12),
            num_return_sequences=n_candidates,
            length_penalty=config.get("length_penalty", 1.0),
            repetition_penalty=config.get("repetition_penalty", 1.2),
            no_repeat_ngram_size=config.get("no_repeat_ngram_size", 3),
            max_length=config.get("max_length", 100),
            early_stopping=True,
            do_sample=False,
        )
    
    candidates = []
    for output in outputs:
        # ⚠️ 수정: 화자 태그 보존하면서 디코딩
        text = decode_with_speaker_tags(tokenizer, output)
        text = postprocess_summary(text)
        candidates.append(text)
    
    return candidates

def rerank_by_reference(candidates, reference):
    """Reference와 비교하여 최적 후보 선택 (Oracle)"""
    best_score = -1
    best_candidate = candidates[0]
    
    for cand in candidates:
        scores = compute_morpheme_rouge(cand, reference)
        if scores['rouge-l'] > best_score:
            best_score = scores['rouge-l']
            best_candidate = cand
    
    return best_candidate, best_score

def rerank_by_heuristic(candidates, target_length=85):
    """Heuristic으로 최적 후보 선택 (Test용)"""
    # 길이 기반 선택 (Dev 최적 길이와 가장 가까운 후보)
    top_candidates = candidates[:3]  # 상위 3개 중 선택
    best_candidate = min(top_candidates, key=lambda x: abs(len(x) - target_length))
    return best_candidate

# N-Best Reranking 설정
NBEST_CONFIG = {
    "num_beams": 12,
    "length_penalty": 1.0,
    "repetition_penalty": 1.2,
    "no_repeat_ngram_size": 3,
    "max_length": 100,
}

# Dev에서 N-Best Reranking 효과 측정
print("\n🔬 N-Best Reranking 효과 측정 (50개 샘플)...")

sample_df = dev_df.sample(n=50, random_state=42)
single_beam_scores = []
nbest_scores = []

for idx, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="N-Best"):
    candidates = generate_nbest_candidates(
        eval_model, eval_tokenizer, 
        row['dialogue'], NBEST_CONFIG, n_candidates=8
    )
    
    # 단일 beam (첫 번째 후보)
    single_score = compute_morpheme_rouge(candidates[0], row['summary'])['rouge-l']
    single_beam_scores.append(single_score)
    
    # N-Best Reranking (Oracle)
    _, best_score = rerank_by_reference(candidates, row['summary'])
    nbest_scores.append(best_score)

avg_single = np.mean(single_beam_scores) * 100
avg_nbest = np.mean(nbest_scores) * 100

print(f"\n📊 N-Best Reranking 결과:")
print(f"   단일 Beam:        {avg_single:.2f}")
print(f"   N-Best (Oracle):  {avg_nbest:.2f}")
print(f"   향상:             +{avg_nbest - avg_single:.2f}")
print(f"\n   예상 리더보드:")
print(f"   단일 Beam:        {avg_single * 1.32:.2f}")
print(f"   N-Best (Oracle):  {avg_nbest * 1.32:.2f}")

# 화자 태그 확인
print("\n📋 화자 태그 확인 (N-Best 샘플):")
print(f"  후보 0: {candidates[0][:80]}...")

🎯 Phase 2-B: N-Best Reranking

🔬 N-Best Reranking 효과 측정 (50개 샘플)...


N-Best: 100%|██████████| 50/50 [00:10<00:00,  4.71it/s]


📊 N-Best Reranking 결과:
   단일 Beam:        34.71
   N-Best (Oracle):  39.69
   향상:             +4.98

   예상 리더보드:
   단일 Beam:        45.82
   N-Best (Oracle):  52.40

📋 화자 태그 확인 (N-Best 샘플):
  후보 0: #Person1# 은 독일 엔지니어가 교통 문제를 해결하기 위해 만든 스쿠터를 소개하며, 디자인이 예쁘고 날렵하다고 설명합니다. 또한, 양 손으...


In [43]:
# ============================================================================
# 📤 Phase 2-B 제출: N-Best Reranking 적용 제출 파일 생성
# ============================================================================

print("=" * 80)
print("📤 Phase 2-B: N-Best Reranking 적용 제출 파일 생성")
print("=" * 80)

def generate_submission_with_nbest(model, tokenizer, test_df, nbest_config, output_path, 
                                   n_candidates=5, target_length=85):
    """
    N-Best 후보 생성 후 Heuristic으로 최적 후보 선택하여 제출 파일 생성
    (Test 데이터는 reference가 없으므로 길이 기반 heuristic 사용)
    """
    summaries = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="N-Best 추론"):
        # N-Best 후보 생성
        candidates = generate_nbest_candidates(
            model, tokenizer, 
            row['dialogue'], 
            nbest_config, 
            n_candidates=n_candidates
        )
        
        # Heuristic 기반 최적 후보 선택 (Test용)
        # 전략: 상위 3개 중 target_length에 가장 가까운 후보 선택
        top_candidates = candidates[:min(3, len(candidates))]
        best_candidate = min(top_candidates, key=lambda x: abs(len(x) - target_length))
        
        summaries.append(best_candidate)
    
    # 제출 파일 생성
    submission = pd.DataFrame({
        'fname': test_df['fname'],
        'summary': summaries
    })
    
    submission.to_csv(output_path, index=False)
    
    # 통계 출력
    has_speaker = sum(1 for s in summaries if '#Person' in s)
    avg_len = np.mean([len(s) for s in summaries])
    print(f"✅ 저장 완료: {output_path}")
    print(f"   평균 길이: {avg_len:.1f}자")
    print(f"   화자 태그 포함: {has_speaker}/{len(summaries)}개 ({has_speaker/len(summaries)*100:.1f}%)")
    
    return submission

# N-Best 설정 (Dev 실험 결과 기반)
NBEST_SUBMISSION_CONFIG = {
    "num_beams": 12,
    "length_penalty": 1.0,
    "repetition_penalty": 1.2,
    "no_repeat_ngram_size": 3,
    "max_length": 100,
}

# Dev에서 관찰된 최적 요약 길이 (평균)
# postproc_results나 base_results에서 가져오기
try:
    optimal_target_length = int(postproc_results['avg_len'])
except:
    optimal_target_length = 80  # 기본값

print(f"📌 N-Best 설정: n_candidates=5, target_length={optimal_target_length}")
print(f"📌 생성 설정: {NBEST_SUBMISSION_CONFIG}")

# 제출 파일 생성
print(f"\n🚀 N-Best Reranking 제출 파일 생성 중...")

submission_nbest = generate_submission_with_nbest(
    eval_model, eval_tokenizer, test_df,
    NBEST_SUBMISSION_CONFIG,
    output_path="./prediction/submit_phase2b_nbest.csv",
    n_candidates=5,
    target_length=optimal_target_length
)

# 샘플 확인
print("\n📋 제출 샘플 (N-Best Reranking):")
for i in range(5):
    print(f"\n[Test {i}]")
    summary = submission_nbest['summary'].iloc[i]
    has_tag = "✅" if "#Person" in summary else "❌"
    print(f"  {has_tag} {summary[:100]}...")

📤 Phase 2-B: N-Best Reranking 적용 제출 파일 생성
📌 N-Best 설정: n_candidates=5, target_length=73
📌 생성 설정: {'num_beams': 12, 'length_penalty': 1.0, 'repetition_penalty': 1.2, 'no_repeat_ngram_size': 3, 'max_length': 100}

🚀 N-Best Reranking 제출 파일 생성 중...


N-Best 추론: 100%|██████████| 499/499 [01:41<00:00,  4.93it/s]

✅ 저장 완료: ./prediction/submit_phase2b_nbest.csv
   평균 길이: 74.9자
   화자 태그 포함: 359/499개 (71.9%)

📋 제출 샘플 (N-Best Reranking):

[Test 0]
  ✅ #Person1# 은 Ms. Dawson에게 사내 메모를 작성하고 배포해 달라고 요청합니다....

[Test 1]
  ✅ #Person1# 과 교통 체증으로 인해 대중교통으로 출퇴근하는 것이 더 나은 선택이 될 수 있다고 생각합니다. 또한 출근할 때 차를 타지 않기로 했습니다....

[Test 2]
  ❌ Kate는 Masha와 Hero가 두 달 동안 별거하다가 결국 이혼을 하게 되었다고 Kate에게 이야기한다....

[Test 3]
  ❌ Brian은 파티에서 자신의 생일을 축하하기 위해 준비한 선물로 목걸이를 선물로 받습니다....

[Test 4]
  ✅ #Person1# 과 올림픽 스타디움의 전체 스타디움이 완공될 예정이며, 총 5000석이 있다고 설명합니다....


In [44]:
# ============================================================================
# 🤝 Phase 3: 모델 앙상블 전략
# ============================================================================

print("=" * 80)
print("🤝 Phase 3: 모델 앙상블 전략")
print("=" * 80)

def load_secondary_model(model_name_or_path, device):
    """보조 모델 로드"""
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
    tokenizer.add_special_tokens({
        'additional_special_tokens': loaded_config['tokenizer']['special_tokens']
    })
    
    model = BartForConditionalGeneration.from_pretrained(model_name_or_path)
    model.resize_token_embeddings(len(tokenizer))
    model = model.to(device)
    model.eval()
    
    return model, tokenizer

# 앙상블 추론 함수
def ensemble_inference(models_info, dialogue, gen_config, reference=None):
    """
    여러 모델의 출력 중 최적 선택 (화자 태그 보존)
    models_info: [(model, tokenizer, name), ...]
    """
    all_candidates = []
    
    for model, tokenizer, name in models_info:
        inputs = tokenizer(
            dialogue,
            return_tensors="pt",
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            padding=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                num_beams=gen_config.get("num_beams", 6),
                num_return_sequences=gen_config.get("num_return_sequences", 3),
                length_penalty=gen_config.get("length_penalty", 1.0),
                repetition_penalty=gen_config.get("repetition_penalty", 1.2),
                no_repeat_ngram_size=gen_config.get("no_repeat_ngram_size", 3),
                max_length=gen_config.get("max_length", 100),
                early_stopping=True,
            )
        
        for output in outputs:
            # ⚠️ 수정: 화자 태그 보존하면서 디코딩
            text = decode_with_speaker_tags(tokenizer, output)
            text = postprocess_summary(text)
            all_candidates.append((text, name))
    
    # Reference가 있으면 ROUGE로 최적 선택, 없으면 첫 번째 반환
    if reference:
        best_candidate = all_candidates[0][0]
        best_score = -1
        for cand, name in all_candidates:
            score = compute_morpheme_rouge(cand, reference)['rouge-l']
            if score > best_score:
                best_score = score
                best_candidate = cand
        return best_candidate, best_score
    else:
        # Heuristic: 길이가 중간인 후보 선택
        lengths = [len(c[0]) for c in all_candidates]
        median_len = np.median(lengths)
        best_candidate = min(all_candidates, key=lambda x: abs(len(x[0]) - median_len))
        return best_candidate[0], None

print("""
📋 앙상블 전략:
1. digit82/kobart-summarization - 리더보드 최고 성능 모델
2. gogamza/kobart-base-v2 - 보완적 특성

앙상블 방법:
- 각 모델에서 N-best 후보 생성
- Dev 데이터 기준 ROUGE로 최적 후보 선택
- Test에서는 길이/다양성 기반 heuristic 적용

⚠️ 화자 태그(#Person1#, #Person2#)는 보존됩니다!
""")

🤝 Phase 3: 모델 앙상블 전략

📋 앙상블 전략:
1. digit82/kobart-summarization - 리더보드 최고 성능 모델
2. gogamza/kobart-base-v2 - 보완적 특성

앙상블 방법:
- 각 모델에서 N-best 후보 생성
- Dev 데이터 기준 ROUGE로 최적 후보 선택
- Test에서는 길이/다양성 기반 heuristic 적용

⚠️ 화자 태그(#Person1#, #Person2#)는 보존됩니다!



In [45]:
# ============================================================================
# 🤝 Phase 3: 모델 앙상블 제출 파일 생성
# ============================================================================

print("=" * 80)
print("🤝 Phase 3: 모델 앙상블 제출 파일 생성")
print("=" * 80)

# 앙상블 설정
ENSEMBLE_CONFIG = {
    "primary_model": "./prediction_kobart_v1/checkpoint-4674",  # digit82 기반 fine-tuned
    "secondary_model": "gogamza/kobart-base-v2",                # 보조 모델
    "gen_config": {
        "num_beams": 6,
        "num_return_sequences": 3,
        "length_penalty": 1.0,
        "repetition_penalty": 1.2,
        "no_repeat_ngram_size": 3,
        "max_length": 100,
    },
    "selection_method": "median_length"  # or "first_primary" or "longest"
}

def generate_ensemble_submission(test_df, models_info, gen_config, output_path, selection_method="median_length"):
    """앙상블 모델로 제출 파일 생성"""
    
    print(f"\n📊 앙상블 모델 제출 파일 생성 시작...")
    print(f"  - 모델 수: {len(models_info)}")
    print(f"  - 선택 방법: {selection_method}")
    print(f"  - 테스트 샘플 수: {len(test_df)}")
    
    results = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="앙상블 추론 중"):
        dialogue = row['dialogue']
        all_candidates = []
        
        # 각 모델에서 후보 생성
        for model, tokenizer, name in models_info:
            inputs = tokenizer(
                dialogue,
                return_tensors="pt",
                max_length=loaded_config['tokenizer']['encoder_max_len'],
                truncation=True,
                padding=True
            ).to(device)
            
            with torch.no_grad():
                outputs = model.generate(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    num_beams=gen_config.get("num_beams", 6),
                    num_return_sequences=gen_config.get("num_return_sequences", 3),
                    length_penalty=gen_config.get("length_penalty", 1.0),
                    repetition_penalty=gen_config.get("repetition_penalty", 1.2),
                    no_repeat_ngram_size=gen_config.get("no_repeat_ngram_size", 3),
                    max_length=gen_config.get("max_length", 100),
                    early_stopping=True,
                )
            
            # 화자 태그 보존하면서 디코딩
            for output in outputs:
                text = decode_with_speaker_tags(tokenizer, output)
                text = postprocess_summary(text)
                all_candidates.append({
                    "text": text,
                    "model": name,
                    "length": len(text)
                })
        
        # 선택 방법에 따라 최적 후보 선택
        if selection_method == "median_length":
            # 중간 길이 후보 선택
            lengths = [c["length"] for c in all_candidates]
            median_len = np.median(lengths)
            best = min(all_candidates, key=lambda x: abs(x["length"] - median_len))
        elif selection_method == "first_primary":
            # Primary 모델의 첫 번째 후보
            best = all_candidates[0]
        elif selection_method == "longest":
            # 가장 긴 후보 선택
            best = max(all_candidates, key=lambda x: x["length"])
        elif selection_method == "shortest":
            # 가장 짧은 후보 선택 (과적합 방지)
            best = min(all_candidates, key=lambda x: x["length"])
        else:
            best = all_candidates[0]
        
        results.append({
            "fname": row['fname'],
            "summary": best["text"]
        })
    
    # 제출 파일 생성
    submission_df = pd.DataFrame(results)
    submission_df.to_csv(output_path, index=False)
    
    # 통계 출력
    summary_lengths = [len(r["summary"]) for r in results]
    print(f"\n✅ 제출 파일 저장 완료: {output_path}")
    print(f"  - 총 샘플 수: {len(results)}")
    print(f"  - 요약 평균 길이: {np.mean(summary_lengths):.1f}자")
    print(f"  - 요약 최소 길이: {min(summary_lengths)}자")
    print(f"  - 요약 최대 길이: {max(summary_lengths)}자")
    
    return submission_df

def generate_single_model_submission(test_df, model, tokenizer, model_name, gen_config, output_path):
    """단일 모델로 제출 파일 생성 (앙상블 비교용)"""
    
    print(f"\n📊 단일 모델 ({model_name}) 제출 파일 생성...")
    
    results = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc=f"{model_name} 추론 중"):
        dialogue = row['dialogue']
        
        inputs = tokenizer(
            dialogue,
            return_tensors="pt",
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            padding=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                num_beams=gen_config.get("num_beams", 6),
                length_penalty=gen_config.get("length_penalty", 1.0),
                repetition_penalty=gen_config.get("repetition_penalty", 1.2),
                no_repeat_ngram_size=gen_config.get("no_repeat_ngram_size", 3),
                max_length=gen_config.get("max_length", 100),
                early_stopping=True,
            )
        
        summary = decode_with_speaker_tags(tokenizer, outputs[0])
        summary = postprocess_summary(summary)
        
        results.append({
            "fname": row['fname'],
            "summary": summary
        })
    
    submission_df = pd.DataFrame(results)
    submission_df.to_csv(output_path, index=False)
    print(f"✅ 저장 완료: {output_path}")
    
    return submission_df

# =============================================================================
# 제출 파일 생성 실행
# =============================================================================

print("\n" + "=" * 60)
print("🚀 Phase 3 앙상블 제출 파일 생성 시작")
print("=" * 60)

# 방법 1: Primary 모델만 사용 (baseline - 현재 모델)
print("\n[1] Primary 모델 (digit82 fine-tuned) 단독 제출 파일...")
submission_primary = generate_single_model_submission(
    test_df=test_df,
    model=eval_model,
    tokenizer=eval_tokenizer,
    model_name="digit82-finetuned",
    gen_config=ENSEMBLE_CONFIG["gen_config"],
    output_path="./prediction/submit_phase3_primary.csv"
)

# 방법 2: 앙상블 시도 (secondary 모델 로드 가능 시)
try:
    print("\n[2] Secondary 모델 로드 시도 (gogamza/kobart-base-v2)...")
    secondary_model, secondary_tokenizer = load_secondary_model(
        ENSEMBLE_CONFIG["secondary_model"], device
    )
    print("✅ Secondary 모델 로드 성공!")
    
    # 앙상블 모델 정보
    models_info = [
        (eval_model, eval_tokenizer, "digit82-finetuned"),
        (secondary_model, secondary_tokenizer, "gogamza-base-v2"),
    ]
    
    # 앙상블 제출 파일 생성 (median_length 방식)
    print("\n[3] 앙상블 제출 파일 생성 (median_length 선택)...")
    submission_ensemble = generate_ensemble_submission(
        test_df=test_df,
        models_info=models_info,
        gen_config=ENSEMBLE_CONFIG["gen_config"],
        output_path="./prediction/submit_phase3_ensemble.csv",
        selection_method="median_length"
    )
    
    # 메모리 정리
    del secondary_model, secondary_tokenizer
    torch.cuda.empty_cache()
    print("\n🧹 Secondary 모델 메모리 해제 완료")
    
except Exception as e:
    print(f"\n⚠️ Secondary 모델 로드 실패: {e}")
    print("   Primary 모델만 사용한 제출 파일을 사용하세요.")
    submission_ensemble = None

# =============================================================================
# 결과 요약
# =============================================================================

print("\n" + "=" * 60)
print("📋 Phase 3 제출 파일 생성 완료")
print("=" * 60)

print("""
생성된 제출 파일:
1. ./prediction/submit_phase3_primary.csv
   - Primary 모델 (digit82 fine-tuned) 단독
   - 앙상블 baseline으로 사용

2. ./prediction/submit_phase3_ensemble.csv (Secondary 모델 로드 성공 시)
   - digit82-finetuned + gogamza-base-v2 앙상블
   - median_length 방식으로 최적 후보 선택

📌 리더보드 제출 순서 권장:
   1) submit_phase3_primary.csv → 단일 모델 baseline
   2) submit_phase3_ensemble.csv → 앙상블 효과 확인
""")

🤝 Phase 3: 모델 앙상블 제출 파일 생성

🚀 Phase 3 앙상블 제출 파일 생성 시작

[1] Primary 모델 (digit82 fine-tuned) 단독 제출 파일...

📊 단일 모델 (digit82-finetuned) 제출 파일 생성...


digit82-finetuned 추론 중: 100%|██████████| 499/499 [01:28<00:00,  5.64it/s]



✅ 저장 완료: ./prediction/submit_phase3_primary.csv

[2] Secondary 모델 로드 시도 (gogamza/kobart-base-v2)...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ Secondary 모델 로드 성공!

[3] 앙상블 제출 파일 생성 (median_length 선택)...

📊 앙상블 모델 제출 파일 생성 시작...
  - 모델 수: 2
  - 선택 방법: median_length
  - 테스트 샘플 수: 499


앙상블 추론 중: 100%|██████████| 499/499 [05:44<00:00,  1.45it/s]


✅ 제출 파일 저장 완료: ./prediction/submit_phase3_ensemble.csv
  - 총 샘플 수: 499
  - 요약 평균 길이: 82.1자
  - 요약 최소 길이: 28자
  - 요약 최대 길이: 222자

🧹 Secondary 모델 메모리 해제 완료

📋 Phase 3 제출 파일 생성 완료

생성된 제출 파일:
1. ./prediction/submit_phase3_primary.csv
   - Primary 모델 (digit82 fine-tuned) 단독
   - 앙상블 baseline으로 사용

2. ./prediction/submit_phase3_ensemble.csv (Secondary 모델 로드 성공 시)
   - digit82-finetuned + gogamza-base-v2 앙상블
   - median_length 방식으로 최적 후보 선택

📌 리더보드 제출 순서 권장:
   1) submit_phase3_primary.csv → 단일 모델 baseline
   2) submit_phase3_ensemble.csv → 앙상블 효과 확인



In [47]:
# ============================================================================
# 🎓 Phase 4: Knowledge Distillation 설정
# ============================================================================

print("=" * 80)
print("🎓 Phase 4: Knowledge Distillation 설정")
print("=" * 80)

DISTILL_CONFIG = {
    "teacher_model": "digit82/kobart-summarization",  # Teacher (더 큰/좋은 모델)
    "student_model": "gogamza/kobart-base-v2",        # Student (학습할 모델)
    "output_dir": "./results_distillation",
    "gold_ratio": 0.5,  # Gold label 50%, Teacher pseudo-label 50%
    "training": {
        "num_train_epochs": 10,
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 16,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "fp16": True,
        "label_smoothing_factor": 0.1,
    }
}

class DistillationDataset(Dataset):
    """Gold label과 Teacher pseudo-label을 혼합한 데이터셋"""
    
    def __init__(self, dialogues, gold_labels, teacher_labels, tokenizer, max_enc_len, max_dec_len, gold_ratio=0.5):
        self.dialogues = dialogues
        self.gold_labels = gold_labels
        self.teacher_labels = teacher_labels
        self.tokenizer = tokenizer
        self.max_enc_len = max_enc_len
        self.max_dec_len = max_dec_len
        
        # 각 샘플에 대해 gold/teacher 중 하나 선택
        self.selected_labels = []
        for i in range(len(dialogues)):
            if random.random() < gold_ratio:
                self.selected_labels.append(gold_labels[i])
            else:
                self.selected_labels.append(teacher_labels[i])
    
    def __len__(self):
        return len(self.dialogues)
    
    def __getitem__(self, idx):
        dialogue = self.dialogues[idx]
        summary = self.selected_labels[idx]
        
        # 인코더 입력
        encoder_inputs = self.tokenizer(
            dialogue,
            max_length=self.max_enc_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        # 디코더 레이블
        decoder_inputs = self.tokenizer(
            summary,
            max_length=self.max_dec_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        labels = decoder_inputs['input_ids'].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return {
            'input_ids': encoder_inputs['input_ids'].squeeze(),
            'attention_mask': encoder_inputs['attention_mask'].squeeze(),
            'labels': labels,
        }

def generate_teacher_pseudo_labels(teacher_model, teacher_tokenizer, dialogues, gen_config, batch_size=16):
    """Teacher 모델로 pseudo-label 생성 (화자 태그 보존)"""
    pseudo_labels = []
    
    for i in tqdm(range(0, len(dialogues), batch_size), desc="Teacher 추론"):
        batch = dialogues[i:i+batch_size]
        
        inputs = teacher_tokenizer(
            batch,
            return_tensors="pt",
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            padding=True
        ).to(device)
        
        with torch.no_grad():
            outputs = teacher_model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                **gen_config
            )
        
        for output in outputs:
            # ⚠️ 수정: 화자 태그 보존하면서 디코딩
            text = decode_with_speaker_tags(teacher_tokenizer, output)
            text = postprocess_summary(text)
            pseudo_labels.append(text)
    
    return pseudo_labels

print("""
📋 Knowledge Distillation 파이프라인:

Step 1: Teacher 모델 (digit82)로 Train 데이터 pseudo-label 생성
Step 2: Gold label 50% + Teacher pseudo-label 50% 혼합 데이터셋 구축
Step 3: Student 모델 (gogamza) 학습
Step 4: 학습된 Student로 추론 및 평가

💡 핵심 아이디어:
- Teacher의 "스타일"과 "선택"을 Student가 학습
- Gold label만 사용하는 것보다 다양한 표현 학습 가능

⚠️ 화자 태그(#Person1#, #Person2#)는 보존됩니다!
""")

🎓 Phase 4: Knowledge Distillation 설정

📋 Knowledge Distillation 파이프라인:

Step 1: Teacher 모델 (digit82)로 Train 데이터 pseudo-label 생성
Step 2: Gold label 50% + Teacher pseudo-label 50% 혼합 데이터셋 구축
Step 3: Student 모델 (gogamza) 학습
Step 4: 학습된 Student로 추론 및 평가

💡 핵심 아이디어:
- Teacher의 "스타일"과 "선택"을 Student가 학습
- Gold label만 사용하는 것보다 다양한 표현 학습 가능

⚠️ 화자 태그(#Person1#, #Person2#)는 보존됩니다!



In [48]:
# ============================================================================
# 📤 최종 제출 파일 생성 (최적 설정 적용) - 화자 태그 보존!
# ============================================================================

print("=" * 80)
print("📤 최종 제출 파일 생성 (화자 태그 보존)")
print("=" * 80)

def generate_final_submission(model, tokenizer, test_df, gen_config, output_path):
    """최종 제출 파일 생성 (화자 태그 보존)"""
    summaries = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="추론"):
        inputs = tokenizer(
            row['dialogue'],
            return_tensors="pt",
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            padding=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                **gen_config
            )
        
        # ⚠️ 핵심 수정: 화자 태그 보존하면서 디코딩!
        pred = decode_with_speaker_tags(tokenizer, outputs[0])
        pred = postprocess_summary(pred)
        summaries.append(pred)
    
    # 제출 파일 생성
    submission = pd.DataFrame({
        'fname': test_df['fname'],
        'summary': summaries
    })
    
    submission.to_csv(output_path, index=False)
    print(f"✅ 저장 완료: {output_path}")
    print(f"   평균 길이: {np.mean([len(s) for s in summaries]):.1f}자")
    
    # 화자 태그 포함 여부 확인
    has_speaker_tags = sum(1 for s in summaries if '#Person' in s)
    print(f"   화자 태그 포함: {has_speaker_tags}/{len(summaries)}개 ({has_speaker_tags/len(summaries)*100:.1f}%)")
    
    return submission

# Test 데이터 로드
test_df = pd.read_csv("./data/test.csv")
print(f"📊 Test 데이터: {len(test_df)}개")

# 최적 설정 (Grid Search 결과 기반)
FINAL_GEN_CONFIG = {
    "num_beams": 7,
    "length_penalty": 1.0,
    "repetition_penalty": 1.2,
    "no_repeat_ngram_size": 3,
    "max_length": 100,
    "early_stopping": True,
}

# 제출 파일 생성
print("\n🚀 제출 파일 생성 중...")
submission = generate_final_submission(
    eval_model, eval_tokenizer, test_df, 
    FINAL_GEN_CONFIG, 
    "./prediction/submit_optimized_v2.csv"
)

# 샘플 출력 (화자 태그 확인)
print("\n📋 제출 샘플 (화자 태그 확인):")
for i in range(5):
    print(f"\n[Test {i}]")
    summary = submission['summary'].iloc[i]
    has_tag = "✅" if "#Person" in summary else "❌"
    print(f"  {has_tag} {summary[:100]}...")

# 기존 output_k1.csv와 비교
print("\n📋 기존 결과(output_k1.csv)와 비교:")
old_output = pd.read_csv("./prediction/output_k1.csv")
for i in range(3):
    print(f"\n[Test {i}]")
    print(f"  기존: {old_output['summary'].iloc[i][:80]}...")
    print(f"  새로: {submission['summary'].iloc[i][:80]}...")

📤 최종 제출 파일 생성 (화자 태그 보존)
📊 Test 데이터: 499개

🚀 제출 파일 생성 중...


추론: 100%|██████████| 499/499 [01:31<00:00,  5.45it/s]

✅ 저장 완료: ./prediction/submit_optimized_v2.csv
   평균 길이: 75.0자
   화자 태그 포함: 359/499개 (71.9%)

📋 제출 샘플 (화자 태그 확인):

[Test 0]
  ❌ Ms. Dawson은 사내 메모를 작성하고 배포하기 위해 부서장에게 요청합니다....

[Test 1]
  ✅ #Person1# 과 교통 체증으로 인해 대중교통으로 출퇴근하는 것이 더 나은 선택이 될 수 있다고 생각합니다. 또한, 출근할 때 차를 타지 않기로 했습니다....

[Test 2]
  ❌ Kate는 Masha와 Hero가 두 달 동안 별거하다가 결국 이혼을 하게 되었다고 전한다....

[Test 3]
  ✅ #Person1# 은 Brian의 생일을 축하하기 위해 선물을 준비했습니다....

[Test 4]
  ✅ #Person1# 과 올림픽 스타디움의 전체 스타디움은 6월에 완공될 예정이며, 총 5000석이 있다....

📋 기존 결과(output_k1.csv)와 비교:

[Test 0]
  기존:  #Person1# 은 Ms. Dawson에게 이메일과 공식 메모만 제한된다고 알립니다. 이는 사내 통신에만 적용되며, 외부 통신에도 적용됩니다...
  새로: Ms. Dawson은 사내 메모를 작성하고 배포하기 위해 부서장에게 요청합니다....

[Test 1]
  기존:  #Person2# 는 #Person1# 에게 출퇴근 시간에 교통체증을 피하기 위해 대중교통을 이용하는 것이 더 나은 선택이라고 조언합니다. #...
  새로: #Person1# 과 교통 체증으로 인해 대중교통으로 출퇴근하는 것이 더 나은 선택이 될 수 있다고 생각합니다. 또한, 출근할 때 차를 타지 않...

[Test 2]
  기존:  #Person1# 은 Kate에게 Masha와 Hero가 두 달 동안 별거하다가 결국 이혼했다고 전합니다. 그들은 자녀 양육에 대해 이야기합니...
  새로: Kate는 Masha와 Hero가 두 달 동안 별거하다가 결국 이

# 📏 요약문 길이 최적화 (Phase 7)

### 문제점:
- 현재 생성 요약문 평균 길이: **75.0자**
- 학습 데이터 정답 요약문 평균 길이: **85.8자**  
- 차이: **약 10.8자 부족**

### 해결 방안:
1. `max_new_tokens` 증가 (60/80/100 → 80/100/128)
2. `min_length` 파라미터 추가 (최소 길이 보장)
3. `length_penalty` 증가 (긴 요약 유도, 1.0 → 1.2~1.5)

In [50]:
# ============================================================================
# Phase 7: 요약문 길이 최적화 버전
# ============================================================================

# 개선된 길이 기반 추론 설정 (더 긴 요약 생성)
LENGTH_OPTIMIZED_INFERENCE_CONFIG = {
    # 짧은 대화 (500자 이하) - 기존 60 → 80
    "short": {
        "char_threshold": 500,
        "max_new_tokens": 80,
        "min_length": 30,
        "num_beams": 4,
        "length_penalty": 1.3,
        "no_repeat_ngram_size": 2,
    },
    # 중간 대화 (500~1500자) - 기존 80 → 100
    "medium": {
        "char_threshold": 1500,
        "max_new_tokens": 100,
        "min_length": 40,
        "num_beams": 5,
        "length_penalty": 1.4,
        "no_repeat_ngram_size": 2,
    },
    # 긴 대화 (1500자 초과) - 기존 100 → 128
    "long": {
        "char_threshold": float('inf'),
        "max_new_tokens": 128,
        "min_length": 50,
        "num_beams": 6,
        "length_penalty": 1.5,
        "no_repeat_ngram_size": 3,
    },
}

# 주제별 최소 길이 설정 추가
TOPIC_LENGTH_CONFIG = {
    "medical": {"max_new_tokens": 128, "min_length": 50, "length_penalty": 1.5},
    "insurance": {"max_new_tokens": 128, "min_length": 50, "length_penalty": 1.5},
    "finance": {"max_new_tokens": 110, "min_length": 45, "length_penalty": 1.4},
    "travel": {"max_new_tokens": 100, "min_length": 40, "length_penalty": 1.3},
    "default": {"max_new_tokens": 90, "min_length": 35, "length_penalty": 1.2},
}

def get_length_category_v2(dialogue_text):
    """대화 길이 카테고리 반환 (v2)"""
    length = len(dialogue_text)
    
    if length <= LENGTH_OPTIMIZED_INFERENCE_CONFIG["short"]["char_threshold"]:
        return "short"
    elif length <= LENGTH_OPTIMIZED_INFERENCE_CONFIG["medium"]["char_threshold"]:
        return "medium"
    else:
        return "long"

def get_dynamic_inference_config_v2(dialogue_text, use_topic=True, use_length=True):
    """
    개선된 동적 추론 설정 반환 (길이 최적화 버전)
    - min_length 추가
    - length_penalty 증가
    """
    config = {
        "max_new_tokens": 90,
        "min_length": 35,
        "num_beams": 5,
        "length_penalty": 1.2,
        "no_repeat_ngram_size": 2,
    }
    
    # 1. 길이 기반 설정
    if use_length:
        length_cat = get_length_category_v2(dialogue_text)
        length_config = LENGTH_OPTIMIZED_INFERENCE_CONFIG[length_cat]
        config.update({
            "max_new_tokens": length_config["max_new_tokens"],
            "min_length": length_config["min_length"],
            "num_beams": length_config["num_beams"],
            "length_penalty": length_config["length_penalty"],
            "no_repeat_ngram_size": length_config["no_repeat_ngram_size"],
        })
    
    # 2. 주제 기반 설정 (길이보다 우선)
    if use_topic:
        topic = detect_topic(dialogue_text)
        if topic != "default":
            topic_config = TOPIC_LENGTH_CONFIG[topic]
            config["max_new_tokens"] = max(config["max_new_tokens"], topic_config["max_new_tokens"])
            config["min_length"] = max(config["min_length"], topic_config["min_length"])
            config["length_penalty"] = max(config["length_penalty"], topic_config["length_penalty"])
    
    return config

print("=" * 60)
print("📏 길이 최적화 버전 추론 설정 (Phase 7)")
print("=" * 60)
print("\n기존 vs 개선 비교:")
print(f"{'카테고리':<10} {'기존 max_tokens':<15} {'개선 max_tokens':<15} {'min_length':<12} {'length_penalty':<14}")
print("-" * 66)

old_config = {
    "short": {"max_new_tokens": 60, "length_penalty": 1.0},
    "medium": {"max_new_tokens": 80, "length_penalty": 1.1},
    "long": {"max_new_tokens": 100, "length_penalty": 1.2},
}

for cat in ["short", "medium", "long"]:
    new_cfg = LENGTH_OPTIMIZED_INFERENCE_CONFIG[cat]
    old_cfg = old_config[cat]
    print(f"{cat:<10} {old_cfg['max_new_tokens']:<15} {new_cfg['max_new_tokens']:<15} {new_cfg['min_length']:<12} {new_cfg['length_penalty']:<14}")

📏 길이 최적화 버전 추론 설정 (Phase 7)

기존 vs 개선 비교:
카테고리       기존 max_tokens   개선 max_tokens   min_length   length_penalty
------------------------------------------------------------------
short      60              80              30           1.3           
medium     80              100             40           1.4           
long       100             128             50           1.5           


In [51]:
# ============================================================================
# Phase 7: 길이 최적화 inference 함수
# ============================================================================

def inference_length_optimized(config):
    """
    길이 최적화 버전 추론 함수
    - min_length 파라미터 추가
    - length_penalty 증가
    - max_new_tokens 증가
    """
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-' * 10, f'device : {device}', '-' * 10)

    generate_model, tokenizer = load_tokenizer_and_model_for_test(config, device)

    # 설정 로드
    prompt_config = config.get('prompt', {
        "use_prompt": False,
        "prompt_style": "none",
        "use_turn_separator": False,
        "turn_separator": "[SEP]",
        "tfidf_top_k": 5,
    })
    
    preprocess_config = config.get('preprocess', {
        "normalize_slang": False,
    })
    
    regex_pattern_config = config.get('regex_pattern', {
        "use_speaker_count": True,
        "use_pii_instruction": True,
        "show_pii_examples": True,
    })
    
    # 5-1. TF-IDF extractor 사용 여부 확인
    use_tfidf = prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']
    tfidf_ext = tfidf_extractor if use_tfidf else None
    
    print(f"\n📋 추론 설정:")
    print(f"   - 프롬프트 스타일: {prompt_config.get('prompt_style', 'none')}")
    print(f"   - 슬랭 정규화: {preprocess_config.get('normalize_slang', False)}")
    print(f"   - 길이 최적화: ✅ (min_length + length_penalty 증가)")
    
    preprocessor = Preprocess(
        config['tokenizer']['bos_token'], 
        config['tokenizer']['eos_token'],
        prompt_config=prompt_config,
        preprocess_config=preprocess_config,
        tfidf_extractor=tfidf_ext,
        regex_pattern_config=regex_pattern_config
    )

    data_path = config['general']['data_path']
    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
    
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    
    # 길이 최적화: 동적 추론 사용
    print("\n🚀 길이 최적화 모드로 생성 시작...")
    sample_idx = 0
    
    with torch.no_grad():
        for item in tqdm(dataloader):
            batch_size = len(item['ID'])
            text_ids.extend(item['ID'])
            
            for i in range(batch_size):
                # 개별 샘플에 대한 길이 최적화 설정 계산
                dialogue_text = test_data.iloc[sample_idx]['dialogue']
                dynamic_cfg = get_dynamic_inference_config_v2(dialogue_text, use_topic=True, use_length=True)
                
                # 단일 샘플 생성 (min_length 추가!)
                single_input_ids = item['input_ids'][i:i+1].to(device)
                generated_ids = generate_model.generate(
                    input_ids=single_input_ids,
                    no_repeat_ngram_size=dynamic_cfg['no_repeat_ngram_size'],
                    early_stopping=config['inference']['early_stopping'],
                    max_length=dynamic_cfg['max_new_tokens'],
                    min_length=dynamic_cfg['min_length'],  # 최소 길이 추가!
                    num_beams=dynamic_cfg['num_beams'],
                    length_penalty=dynamic_cfg['length_penalty'],  # 증가된 length_penalty
                )
                
                # 화자 태그 보존하면서 디코딩
                result = decode_with_speaker_tags(tokenizer, generated_ids[0])
                summary.append(result)
                sample_idx += 1

    # 후처리 적용 (띄어쓰기 교정)
    preprocessed_summary = [postprocess_summary(s) for s in summary]

    output = pd.DataFrame({
        "fname": test_data['fname'],
        "summary": preprocessed_summary,
    })
    
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)
    
    # 길이 통계 출력
    lengths = output['summary'].str.len()
    print(f"\n📊 생성 요약문 길이 통계:")
    print(f"   평균: {lengths.mean():.1f}자")
    print(f"   최소: {lengths.min()}자")
    print(f"   최대: {lengths.max()}자")
    print(f"   중앙값: {lengths.median():.1f}자")

    return output

print("✅ inference_length_optimized() 함수 정의 완료")

✅ inference_length_optimized() 함수 정의 완료


================================================================================
📤 최종 제출 파일 생성 (화자 태그 보존)
================================================================================
📊 Test 데이터: 499개

🚀 제출 파일 생성 중...
추론: 100%|██████████| 499/499 [01:32<00:00,  5.41it/s]
✅ 저장 완료: ./prediction/submit_optimized_v2.csv
   평균 길이: 81.7자
   화자 태그 포함: 434/499개 (87.0%)

📋 제출 샘플 (화자 태그 확인):

[Test 0]
  ✅ #Person1# 은 Ms. Dawson에게 사내 메모를 작성하고 배포해 달라고 요청합니다....

[Test 1]
  ✅ #Person2# 는 #Person1#에게 출퇴근 시간에 교통 체증을 피하기 위해 대중교통을 이용하라고 제안합니다. #Person1# 은 대중교통을 이용하는 것이 환경에도 더 좋다...

[Test 2]
  ✅ #Person1# 은 Kate에게 Masha와 Hero가 두 달 동안 별거하다가 결국 이혼했다고 전한다....

[Test 3]
  ✅ #Person1# 은 Brian에게 생일 선물로 목걸이를 주며, 파티에서 함께 춤을 추자고 제안합니다....

[Test 4]
  ✅ #Person1# 과 #Person2# 는 올림픽 공원의 크기와 시설에 대해 이야기하고 있습니다....

📋 기존 결과(output_k1.csv)와 비교:

[Test 0]
  기존:  #Person1# 은 Ms. Dawson에게 이메일과 공식 메모만 제한된다고 알립니다. 이는 사내 통신에만 적용되며, 외부 통신에도 적용됩니다...
...

[Test 2]
  기존:  #Person1# 은 Kate에게 Masha와 Hero가 두 달 동안 별거하다가 결국 이혼했다고 전합니다. 그들은 자녀 양육에 대해 이야기합니...
  새로: #Person1# 은 Kate에게 Masha와 Hero가 두 달 동안 별거하다가 결국 이혼했다고 전한다....
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...


In [52]:
# ============================================================================
# ▶ Phase 1-B 제출 파일 생성 (BASE_GEN_CONFIG + 후처리)
# ============================================================================

print("▶ Phase 1-B 제출 파일 생성: BASE_GEN_CONFIG 적용, 후처리 적용")

import shutil

# BASE_GEN_CONFIG 는 Phase 1-A에서 정의되어 있습니다. 없으면 inference 설정을 사용합니다.
try:
    gen_cfg = BASE_GEN_CONFIG
except NameError:
    gen_cfg = {
        "num_beams": loaded_config['inference']['num_beams'],
        "no_repeat_ngram_size": loaded_config['inference']['no_repeat_ngram_size'],
        "max_length": loaded_config['inference']['generate_max_length'],
        "early_stopping": loaded_config['inference']['early_stopping'],
    }

# 생성 함수가 이미 노트북에 있으므로 재사용합니다.
# 파일명: Phase1-B 전용
out_path = "./prediction/submit_phase1b.csv"

submission_phase1b = generate_final_submission(eval_model, eval_tokenizer, test_df, gen_cfg, out_path)

# 추가로 요청자 혼동을 줄이기 위해 같은 내용을 '_v2_.csv' 라는 이름으로도 저장합니다.
shutil.copy(out_path, "./prediction/submit_optimized_v2_.csv")
print(f"▶ 두 파일 생성: {out_path}, ./prediction/submit_optimized_v2_.csv")

▶ Phase 1-B 제출 파일 생성: BASE_GEN_CONFIG 적용, 후처리 적용


추론: 100%|██████████| 499/499 [01:23<00:00,  5.98it/s]

✅ 저장 완료: ./prediction/submit_phase1b.csv
   평균 길이: 73.4자
   화자 태그 포함: 360/499개 (72.1%)
▶ 두 파일 생성: ./prediction/submit_phase1b.csv, ./prediction/submit_optimized_v2_.csv


In [53]:
# ============================================================================
# 🔬 Phase 5: TF-IDF 키워드 프롬프트 + 동적 추론 실험
# ============================================================================
# 목표: TF-IDF 기반 핵심 키워드를 프롬프트에 추가하고, 대화 길이에 따라 
#       추론 설정을 동적으로 조절하여 성능 향상
# ============================================================================

print("=" * 80)
print("🔬 Phase 5: TF-IDF 키워드 프롬프트 + 동적 추론 실험")
print("=" * 80)

# =============================================================================
# Step 1: TF-IDF Vectorizer 학습 (학습 데이터 기반)
# =============================================================================

print("\n📊 Step 1: TF-IDF Vectorizer 학습 중...")

# Train 데이터 로드 및 대화문 추출
train_df_tfidf = pd.read_csv("./data/train.csv")
train_dialogues = train_df_tfidf['dialogue'].tolist()

# TF-IDF Vectorizer 학습
print(f"   - 학습 데이터 수: {len(train_dialogues)}개")
print(f"   - TF-IDF 설정: min_df={TFIDF_CONFIG['min_df']}, max_df={TFIDF_CONFIG['max_df']}")

# Kiwi 기반 토크나이저로 학습
try:
    tfidf_extractor.fit(train_dialogues)
    print(f"   ✅ TF-IDF Vectorizer 학습 완료")
    print(f"   - Vocabulary 크기: {len(tfidf_extractor.vectorizer.vocabulary_)}개")
except Exception as e:
    print(f"   ❌ TF-IDF 학습 실패: {e}")
    tfidf_extractor = None

# =============================================================================
# Step 2: 샘플 대화문에서 키워드 추출 테스트
# =============================================================================

print("\n🔍 Step 2: TF-IDF 키워드 추출 테스트")

if tfidf_extractor:
    sample_dialogues = train_df_tfidf['dialogue'].head(5).tolist()
    sample_summaries = train_df_tfidf['summary'].head(5).tolist()
    
    for i, (dialogue, summary) in enumerate(zip(sample_dialogues, sample_summaries)):
        keywords = extract_tfidf_keywords(dialogue, tfidf_extractor, top_k=5)
        print(f"\n[Sample {i+1}]")
        print(f"  대화문(일부): {dialogue[:80]}...")
        print(f"  요약문: {summary[:60]}...")
        print(f"  TF-IDF 키워드: {keywords}")

# =============================================================================
# Step 3: 동적 추론 설정 테스트
# =============================================================================

print("\n\n⚙️ Step 3: 동적 추론 설정 테스트")

# Test 데이터의 대화문 길이 분포 확인
test_df_phase5 = pd.read_csv("./data/test.csv")
test_dialogue_lengths = [len(d) for d in test_df_phase5['dialogue'].tolist()]

short_count = sum(1 for l in test_dialogue_lengths if l <= 500)
medium_count = sum(1 for l in test_dialogue_lengths if 500 < l <= 1000)
long_count = sum(1 for l in test_dialogue_lengths if l > 1000)

print(f"\n📏 Test 데이터 대화문 길이 분포:")
print(f"   - Short (≤500자): {short_count}개 ({short_count/len(test_dialogue_lengths)*100:.1f}%)")
print(f"   - Medium (501-1000자): {medium_count}개 ({medium_count/len(test_dialogue_lengths)*100:.1f}%)")
print(f"   - Long (>1000자): {long_count}개 ({long_count/len(test_dialogue_lengths)*100:.1f}%)")

# 각 길이 범주별 동적 설정 확인
print("\n🔧 길이별 동적 추론 설정:")
for test_len in [300, 700, 1500]:
    config = get_dynamic_inference_config(test_len, DYNAMIC_INFERENCE_CONFIG)
    print(f"   - {test_len}자 → max_tokens={config['max_new_tokens']}, num_beams={config['num_beams']}")

# =============================================================================
# Step 4: 다양한 프롬프트 스타일 비교 (Dev 데이터 기준)
# =============================================================================

print("\n\n📝 Step 4: 프롬프트 스타일 별 성능 비교 (Dev 데이터 5샘플)")

# 테스트할 프롬프트 스타일
prompt_styles = [
    {"name": "none", "use_prompt": False, "prompt_style": "none"},
    {"name": "default", "use_prompt": True, "prompt_style": "default"},
    {"name": "speaker_aware", "use_prompt": True, "prompt_style": "speaker_aware"},
    {"name": "keyword (TF-IDF)", "use_prompt": True, "prompt_style": "keyword", "tfidf_top_k": 5},
    {"name": "keyword_aware (Speaker+TF-IDF)", "use_prompt": True, "prompt_style": "keyword_aware", "tfidf_top_k": 5},
]

dev_df_phase5 = pd.read_csv("./data/dev.csv")
sample_dev = dev_df_phase5.head(5)

print(f"\n테스트 샘플 수: {len(sample_dev)}개")

# 각 프롬프트 스타일별 전처리 결과 비교
for style_config in prompt_styles:
    print(f"\n{'='*60}")
    print(f"🔹 프롬프트 스타일: {style_config['name']}")
    print(f"{'='*60}")
    
    # Preprocessor 생성
    preprocessor = Preprocess(
        loaded_config['tokenizer']['bos_token'],
        loaded_config['tokenizer']['eos_token'],
        prompt_config=style_config,
        preprocess_config={"normalize_slang": True},  # 슬랭 정규화 활성화
        tfidf_extractor=tfidf_extractor if 'keyword' in style_config.get('prompt_style', '') else None
    )
    
    # 첫 번째 샘플로 전처리 결과 확인
    sample_dialogue = sample_dev.iloc[0]['dialogue']
    processed = preprocessor.full_preprocess_pipeline(sample_dialogue)
    
    print(f"\n[원본 대화문 (일부)]")
    print(f"  {sample_dialogue[:100]}...")
    print(f"\n[전처리 결과]")
    print(f"  {processed[:150]}...")
    print(f"\n  길이: 원본={len(sample_dialogue)}자 → 전처리={len(processed)}자")

print("\n" + "=" * 80)
print("✅ Phase 5 설정 완료!")
print("=" * 80)
print("""
다음 단계:
1. 위 프롬프트 스타일 중 가장 적합한 것 선택
2. config에 적용하여 추론 실행
3. 제출 파일 생성 및 리더보드 평가
""")

🔬 Phase 5: TF-IDF 키워드 프롬프트 + 동적 추론 실험

📊 Step 1: TF-IDF Vectorizer 학습 중...
   - 학습 데이터 수: 12457개


NameError: name 'TFIDF_CONFIG' is not defined

In [ ]:
# ============================================================================
# 📤 Phase 5 제출 파일 생성: TF-IDF 키워드 + 동적 추론
# ============================================================================

print("=" * 80)
print("📤 Phase 5 제출 파일 생성")
print("=" * 80)

# Phase 5 설정
PHASE5_CONFIG = {
    # 프롬프트 설정
    "prompt": {
        "use_prompt": True,
        "prompt_style": "keyword_aware",  # Speaker + TF-IDF 키워드
        "use_turn_separator": True,
        "turn_separator": " [SEP] ",
        "tfidf_top_k": 5,
    },
    # 구어체 전처리 설정
    "preprocess": {
        "normalize_slang": True,  # 슬랭 정규화 활성화
    },
    # 동적 추론 설정
    "dynamic_inference": {
        "use_dynamic_settings": False,  # 먼저 기본 설정으로 테스트
        "length_thresholds": {
            "short": {"max_chars": 500, "max_new_tokens": 80, "num_beams": 4},
            "medium": {"max_chars": 1000, "max_new_tokens": 100, "num_beams": 5},
            "long": {"max_chars": float('inf'), "max_new_tokens": 120, "num_beams": 6},
        }
    },
    # 생성 설정 (Grid Search 최적값)
    "generation": {
        "num_beams": 7,
        "length_penalty": 1.0,
        "repetition_penalty": 1.2,
        "no_repeat_ngram_size": 3,
        "max_length": 100,
        "early_stopping": True,
    }
}

print("\n📋 Phase 5 설정:")
print(f"   프롬프트 스타일: {PHASE5_CONFIG['prompt']['prompt_style']}")
print(f"   TF-IDF 키워드: {PHASE5_CONFIG['prompt']['tfidf_top_k']}개")
print(f"   슬랭 정규화: {PHASE5_CONFIG['preprocess']['normalize_slang']}")
print(f"   동적 추론: {PHASE5_CONFIG['dynamic_inference']['use_dynamic_settings']}")

# =============================================================================
# 제출 파일 생성 함수
# =============================================================================

def generate_phase5_submission(model, tokenizer, test_df, config, output_path):
    """Phase 5 설정으로 제출 파일 생성"""
    
    device = next(model.parameters()).device
    
    # Preprocessor 생성
    preprocessor = Preprocess(
        loaded_config['tokenizer']['bos_token'],
        loaded_config['tokenizer']['eos_token'],
        prompt_config=config['prompt'],
        preprocess_config=config['preprocess'],
        tfidf_extractor=tfidf_extractor if 'keyword' in config['prompt'].get('prompt_style', '') else None
    )
    
    gen_config = config['generation']
    dynamic_config = config.get('dynamic_inference', {})
    use_dynamic = dynamic_config.get('use_dynamic_settings', False)
    
    summaries = []
    
    # 대화문 길이 계산 (동적 추론용)
    dialogue_lengths = [len(d) for d in test_df['dialogue'].tolist()]
    
    for idx in tqdm(range(len(test_df)), desc="추론"):
        dialogue = test_df.iloc[idx]['dialogue']
        
        # 전처리
        processed_dialogue = preprocessor.full_preprocess_pipeline(dialogue)
        
        # 토큰화
        inputs = tokenizer(
            processed_dialogue,
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            return_tensors='pt'
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 동적 추론 설정
        if use_dynamic:
            dyn_cfg = get_dynamic_inference_config(dialogue_lengths[idx], dynamic_config)
            max_length = dyn_cfg['max_new_tokens']
            num_beams = dyn_cfg['num_beams']
        else:
            max_length = gen_config['max_length']
            num_beams = gen_config['num_beams']
        
        # 생성
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                num_beams=num_beams,
                length_penalty=gen_config['length_penalty'],
                repetition_penalty=gen_config['repetition_penalty'],
                no_repeat_ngram_size=gen_config['no_repeat_ngram_size'],
                max_length=max_length,
                early_stopping=gen_config['early_stopping'],
            )
        
        # 디코딩 (화자 태그 보존)
        summary = decode_with_speaker_tags(tokenizer, generated_ids[0])
        
        # 후처리
        summary = postprocess_summary(summary)
        summaries.append(summary)
    
    # 제출 파일 생성
    submission = pd.DataFrame({
        'fname': test_df['fname'],
        'summary': summaries
    })
    
    submission.to_csv(output_path, index=False)
    print(f"\n✅ 저장 완료: {output_path}")
    print(f"   평균 길이: {np.mean([len(s) for s in summaries]):.1f}자")
    
    # 화자 태그 포함 여부 확인
    has_speaker_tags = sum(1 for s in summaries if '#Person' in s)
    print(f"   화자 태그 포함: {has_speaker_tags}/{len(summaries)}개 ({has_speaker_tags/len(summaries)*100:.1f}%)")
    
    return submission

# =============================================================================
# 제출 파일 생성 실행
# =============================================================================

print("\n🚀 Phase 5 제출 파일 생성 중...")

# Test 데이터 로드
test_df_submit = pd.read_csv("./data/test.csv")
print(f"📊 Test 데이터: {len(test_df_submit)}개")

# Phase 5 제출 파일 생성
submission_phase5 = generate_phase5_submission(
    eval_model, eval_tokenizer, test_df_submit,
    PHASE5_CONFIG,
    "./prediction/submit_phase5_tfidf.csv"
)

# =============================================================================
# 샘플 비교 출력
# =============================================================================

print("\n📋 Phase 5 제출 샘플:")
for i in range(5):
    summary = submission_phase5['summary'].iloc[i]
    has_tag = "✅" if "#Person" in summary else "❌"
    print(f"\n[Test {i}]")
    print(f"  {has_tag} {summary[:120]}...")

# 기존 최적화 제출과 비교
print("\n📋 기존 최적화 제출(submit_optimized_v2.csv)과 비교:")
try:
    old_submission = pd.read_csv("./prediction/submit_optimized_v2.csv")
    for i in range(3):
        print(f"\n[Test {i}]")
        print(f"  기존: {old_submission['summary'].iloc[i][:80]}...")
        print(f"  Phase5: {submission_phase5['summary'].iloc[i][:80]}...")
except:
    print("   기존 제출 파일을 찾을 수 없습니다.")

print("\n" + "=" * 80)
print("✅ Phase 5 제출 파일 생성 완료!")
print("=" * 80)
print("""
📤 제출 파일 목록:
  - ./prediction/submit_phase5_tfidf.csv (TF-IDF 키워드 + Speaker-aware 프롬프트)

🔍 다음 실험 옵션:
  1. 동적 추론 활성화 (use_dynamic_settings=True)
  2. 다른 프롬프트 스타일 테스트 (default, speaker_aware, keyword)
  3. TF-IDF 키워드 수 조절 (tfidf_top_k: 3, 5, 7)
""")

# 📊 전략별 기대 효과 요약

## 🎯 각 전략의 예상 점수 향상

| 단계 | 전략 | 예상 향상폭 | 누적 예상 점수 |
|:---:|:---|:---:|:---:|
| **Phase 1** | 형태소 ROUGE + 후처리 | +0.5~0.8 | 48.8~49.1 |
| **Phase 2-A** | Grid Search 최적화 | +0.3~0.5 | 49.1~49.6 |
| **Phase 2-B** | N-Best Reranking | +0.3~0.8 | 49.4~50.4 |
| **Phase 3** | 모델 앙상블 | +0.5~1.0 | 49.9~51.4 |
| **Phase 4** | Knowledge Distillation | +0.5~1.0 | 50.4~52.4 |

## 🏃 권장 실행 순서

1. **Phase 1**: 기본 평가 및 후처리 효과 측정 (빠름)
2. **Phase 2-A**: Grid Search로 최적 하이퍼파라미터 탐색
3. **Phase 2-B**: N-Best Reranking 적용 (추론 시간 증가)
4. **Phase 3**: 다른 모델과 앙상블 (gogamza 모델 필요)
5. **Phase 4**: Distillation 학습 (시간 소요 높음)

## ⚠️ 주의사항

- **GPU 메모리**: N-Best, 앙상블은 메모리 사용량이 높습니다
- **학습 시간**: Distillation은 추가 학습이 필요합니다
- **조합 효과**: 모든 전략을 조합하면 시너지 효과가 있습니다
- **검증 필수**: 각 단계마다 Dev ROUGE로 효과를 확인하세요

## 🔧 빠른 시작 가이드

```python
# 1. 가장 빠른 개선: 후처리만 적용
submission = postprocess_predictions("./prediction/output_k1.csv")

# 2. 중간 수준 개선: Grid Search + N-Best
best_config = run_grid_search(model, tokenizer, dev_df)
submission = generate_with_nbest(model, tokenizer, test_df, best_config)

# 3. 최고 성능: 앙상블 + Distillation
ensemble_submission = ensemble_inference(model1, model2, tokenizer, test_df)
```

In [ ]:
# ============================================================================
# 🔍 Phase 6: 정규표현식 기반 패턴 활용 전략 실험
# ============================================================================
# 목표: 화자 수 명시(6-1) + PII 마스킹 유지 지시(6-2)로 ROUGE 점수 향상
# ============================================================================

print("=" * 80)
print("🔍 Phase 6: 정규표현식 기반 패턴 활용 전략")
print("=" * 80)

# =============================================================================
# Step 1: Train/Dev 데이터에서 화자 수 & PII 마스킹 분포 분석
# =============================================================================

print("\n📊 Step 1: 데이터 분석")

# 데이터 로드
train_df_phase6 = pd.read_csv("./data/train.csv")
dev_df_phase6 = pd.read_csv("./data/dev.csv")
test_df_phase6 = pd.read_csv("./data/test.csv")

def analyze_dataset_patterns(df, name):
    """데이터셋의 화자 수 및 PII 마스킹 패턴 분석"""
    speaker_counts = []
    pii_counts = []
    pii_types = set()
    
    for dialogue in df['dialogue']:
        analysis = analyze_dialogue_structure(dialogue)
        speaker_counts.append(analysis['num_speakers'])
        pii_counts.append(len(analysis['pii_masks']))
        pii_types.update(analysis['pii_masks'])
    
    print(f"\n📊 {name} 데이터셋 ({len(df)}개):")
    print(f"   화자 수 분포:")
    for num, count in sorted(pd.Series(speaker_counts).value_counts().items()):
        print(f"      {num}명: {count}개 ({count/len(df)*100:.1f}%)")
    
    pii_exists = sum(1 for c in pii_counts if c > 0)
    print(f"   PII 마스킹 존재: {pii_exists}/{len(df)}개 ({pii_exists/len(df)*100:.1f}%)")
    print(f"   PII 마스킹 종류: {sorted(pii_types)}")
    
    return speaker_counts, pii_counts, pii_types

train_speakers, train_pii, train_pii_types = analyze_dataset_patterns(train_df_phase6, "Train")
dev_speakers, dev_pii, dev_pii_types = analyze_dataset_patterns(dev_df_phase6, "Dev")
test_speakers, test_pii, test_pii_types = analyze_dataset_patterns(test_df_phase6, "Test")

# =============================================================================
# Step 2: 6-1, 6-2 프롬프트 생성 예시
# =============================================================================

print("\n" + "=" * 60)
print("📝 Step 2: 6-1, 6-2 프롬프트 생성 예시")
print("=" * 60)

# PII 마스킹이 있는 샘플 찾기
sample_with_pii = None
for i, dialogue in enumerate(train_df_phase6['dialogue']):
    analysis = analyze_dialogue_structure(dialogue)
    if analysis['has_pii_masks']:
        sample_with_pii = (i, dialogue, train_df_phase6['summary'].iloc[i])
        break

if sample_with_pii:
    idx, dialogue, summary = sample_with_pii
    analysis = analyze_dialogue_structure(dialogue)
    
    print(f"\n[샘플 {idx}] PII 마스킹 포함 대화:")
    print(f"  대화문(앞 200자): {dialogue[:200]}...")
    print(f"\n  🔍 대화 구조 분석:")
    print(f"     - 화자 수: {analysis['num_speakers']}명")
    print(f"     - PII 마스킹: {list(analysis['pii_masks'])}")
    
    print(f"\n  📝 6-1 화자 수 프롬프트:")
    speaker_prompt = build_speaker_aware_prompt(analysis['num_speakers'])
    print(f"     \"{speaker_prompt}\"")
    
    print(f"\n  📝 6-2 PII 마스킹 유지 지시:")
    pii_prompt = build_pii_mask_instruction(analysis['pii_masks'], show_examples=True)
    print(f"     \"{pii_prompt}\"")
    
    print(f"\n  📝 통합 프롬프트 (balanced 스타일):")
    full_prompt = build_summary_prompt(
        dialogue,
        prompt_style="balanced",
        use_speaker_count=True,
        use_pii_instruction=True
    )
    print(f"     {full_prompt[:300]}...")
    
    print(f"\n  ✅ 정답 요약문:")
    print(f"     {summary}")

# =============================================================================
# Step 3: 프롬프트 스타일별 비교 실험 (Dev 5샘플)
# =============================================================================

print("\n" + "=" * 60)
print("🧪 Step 3: 프롬프트 스타일별 비교 (Dev 5샘플)")
print("=" * 60)

# 프롬프트 스타일 설정
PROMPT_STYLES_PHASE6 = [
    {
        "name": "baseline (none)",
        "prompt_style": "none",
        "use_speaker_count": False,
        "use_pii_instruction": False,
    },
    {
        "name": "6-1만 (화자 수)",
        "prompt_style": "speaker_aware",
        "use_speaker_count": True,
        "use_pii_instruction": False,
    },
    {
        "name": "6-2만 (PII 유지 지시)",
        "prompt_style": "default",
        "use_speaker_count": False,
        "use_pii_instruction": True,
    },
    {
        "name": "6-1 + 6-2 (balanced)",
        "prompt_style": "balanced",
        "use_speaker_count": True,
        "use_pii_instruction": True,
    },
    {
        "name": "TF-IDF + 6-1 + 6-2 (keyword_aware)",
        "prompt_style": "keyword_aware",
        "use_speaker_count": True,
        "use_pii_instruction": True,
    },
]

# Dev 샘플에서 PII 마스킹 있는 것 우선 선택
dev_samples_phase6 = []
for i, dialogue in enumerate(dev_df_phase6['dialogue']):
    analysis = analyze_dialogue_structure(dialogue)
    if analysis['has_pii_masks'] and len(dev_samples_phase6) < 3:
        dev_samples_phase6.append(i)
# 나머지는 순서대로
for i in range(len(dev_df_phase6)):
    if i not in dev_samples_phase6 and len(dev_samples_phase6) < 5:
        dev_samples_phase6.append(i)

print(f"\n선택된 Dev 샘플: {dev_samples_phase6}")

for sample_idx in dev_samples_phase6[:2]:  # 처음 2개만 상세 출력
    dialogue = dev_df_phase6.iloc[sample_idx]['dialogue']
    summary = dev_df_phase6.iloc[sample_idx]['summary']
    analysis = analyze_dialogue_structure(dialogue)
    
    print(f"\n{'='*60}")
    print(f"[Dev 샘플 {sample_idx}]")
    print(f"  화자: {analysis['num_speakers']}명, PII: {list(analysis['pii_masks'])[:3]}")
    print(f"  정답 요약: {summary[:80]}...")
    
    for style in PROMPT_STYLES_PHASE6[:3]:  # 처음 3개 스타일만 비교
        print(f"\n  🔹 {style['name']}:")
        
        if style['prompt_style'] == "keyword_aware":
            prompted = build_summary_prompt(
                dialogue,
                prompt_style=style['prompt_style'],
                tfidf_extractor=tfidf_extractor,
                top_k=5,
                use_speaker_count=style['use_speaker_count'],
                use_pii_instruction=style['use_pii_instruction']
            )
        else:
            prompted = build_summary_prompt(
                dialogue,
                prompt_style=style['prompt_style'],
                use_speaker_count=style['use_speaker_count'],
                use_pii_instruction=style['use_pii_instruction']
            )
        
        # 프롬프트 부분만 출력 (대화문 제외)
        prompt_only = prompted.split('\n\n')[0] if '\n\n' in prompted else prompted[:200]
        print(f"     프롬프트: {prompt_only[:150]}...")

print("\n" + "=" * 80)
print("✅ Phase 6 분석 완료!")
print("=" * 80)

In [ ]:
# ============================================================================
# 📤 Phase 6 제출 파일 생성: 6-1 + 6-2 전략 적용
# ============================================================================

print("=" * 80)
print("📤 Phase 6 제출 파일 생성")
print("=" * 80)

# Phase 6 설정 (6-1, 6-2 전략 적용)
PHASE6_CONFIG = {
    # 프롬프트 설정 (balanced = 6-1 + 6-2)
    "prompt": {
        "use_prompt": True,
        "prompt_style": "balanced",  # 6-1 + 6-2 통합 스타일
        "use_turn_separator": False,
        "turn_separator": " [SEP] ",
        "tfidf_top_k": 5,
    },
    # 6. 정규표현식 패턴 활용 설정
    "regex_pattern": {
        "use_speaker_count": True,       # 6-1. 화자 수 명시
        "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
        "show_pii_examples": True,       # 마스킹 예시 표시
    },
    # 구어체 전처리 설정
    "preprocess": {
        "normalize_slang": True,
    },
    # 동적 추론 설정
    "dynamic_inference": {
        "use_length_based": False,
        "use_topic_based": False,
    },
    # 생성 설정 (Grid Search 최적값)
    "generation": {
        "num_beams": 7,
        "length_penalty": 1.0,
        "repetition_penalty": 1.2,
        "no_repeat_ngram_size": 3,
        "max_length": 100,
        "early_stopping": True,
    }
}

print("\n📋 Phase 6 설정:")
print(f"   프롬프트 스타일: {PHASE6_CONFIG['prompt']['prompt_style']}")
print(f"   6-1 화자 수 명시: {PHASE6_CONFIG['regex_pattern']['use_speaker_count']}")
print(f"   6-2 PII 마스킹 유지 지시: {PHASE6_CONFIG['regex_pattern']['use_pii_instruction']}")
print(f"   슬랭 정규화: {PHASE6_CONFIG['preprocess']['normalize_slang']}")

# =============================================================================
# 제출 파일 생성 함수
# =============================================================================

def generate_phase6_submission(model, tokenizer, test_df, config, output_path):
    """Phase 6 설정으로 제출 파일 생성 (6-1 + 6-2 전략)"""
    
    device = next(model.parameters()).device
    
    # 6. 정규표현식 패턴 설정
    regex_pattern_config = config.get('regex_pattern', {
        "use_speaker_count": True,
        "use_pii_instruction": True,
        "show_pii_examples": True,
    })
    
    # Preprocessor 생성
    preprocessor = Preprocess(
        loaded_config['tokenizer']['bos_token'],
        loaded_config['tokenizer']['eos_token'],
        prompt_config=config['prompt'],
        preprocess_config=config['preprocess'],
        tfidf_extractor=None,  # balanced 스타일에서는 TF-IDF 미사용
        regex_pattern_config=regex_pattern_config,  # 6. 정규표현식 패턴 설정
    )
    
    gen_config = config['generation']
    
    summaries = []
    pii_preserved_count = 0  # PII 마스킹 보존 카운트
    
    for idx in tqdm(range(len(test_df)), desc="추론"):
        dialogue = test_df.iloc[idx]['dialogue']
        
        # 원본 대화에서 PII 마스킹 추출
        original_pii = get_pii_mask_tokens(dialogue)
        
        # 전처리
        processed_dialogue = preprocessor.full_preprocess_pipeline(dialogue)
        
        # 토큰화
        inputs = tokenizer(
            processed_dialogue,
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            return_tensors='pt'
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 생성
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                num_beams=gen_config['num_beams'],
                length_penalty=gen_config['length_penalty'],
                repetition_penalty=gen_config['repetition_penalty'],
                no_repeat_ngram_size=gen_config['no_repeat_ngram_size'],
                max_length=gen_config['max_length'],
                early_stopping=gen_config['early_stopping'],
            )
        
        # 디코딩 (화자 태그 보존)
        summary = decode_with_speaker_tags(tokenizer, generated_ids[0])
        
        # 후처리
        summary = postprocess_summary(summary)
        
        # PII 마스킹 보존 여부 확인
        if original_pii:
            preserved = all(pii in summary for pii in original_pii)
            if preserved:
                pii_preserved_count += 1
        
        summaries.append(summary)
    
    # 제출 파일 생성
    submission = pd.DataFrame({
        'fname': test_df['fname'],
        'summary': summaries
    })
    
    submission.to_csv(output_path, index=False)
    print(f"\n✅ 저장 완료: {output_path}")
    print(f"   평균 길이: {np.mean([len(s) for s in summaries]):.1f}자")
    
    # 화자 태그 포함 여부 확인
    has_speaker_tags = sum(1 for s in summaries if '#Person' in s)
    print(f"   화자 태그 포함: {has_speaker_tags}/{len(summaries)}개 ({has_speaker_tags/len(summaries)*100:.1f}%)")
    
    # PII 마스킹 보존 비율
    pii_total = sum(1 for d in test_df['dialogue'] if get_pii_mask_tokens(d))
    if pii_total > 0:
        print(f"   PII 마스킹 보존: {pii_preserved_count}/{pii_total}개 ({pii_preserved_count/pii_total*100:.1f}%)")
    
    return submission

# =============================================================================
# 제출 파일 생성 실행
# =============================================================================

print("\n🚀 Phase 6 제출 파일 생성 중...")

# Test 데이터 로드
test_df_phase6_submit = pd.read_csv("./data/test.csv")
print(f"📊 Test 데이터: {len(test_df_phase6_submit)}개")

# Phase 6 제출 파일 생성 (balanced: 6-1 + 6-2)
submission_phase6 = generate_phase6_submission(
    eval_model, eval_tokenizer, test_df_phase6_submit,
    PHASE6_CONFIG,
    "./prediction/submit_phase6_balanced.csv"
)

# =============================================================================
# keyword_aware 스타일로도 생성 (TF-IDF + 6-1 + 6-2)
# =============================================================================

PHASE6_KEYWORD_CONFIG = {
    **PHASE6_CONFIG,
    "prompt": {
        "use_prompt": True,
        "prompt_style": "keyword_aware",  # TF-IDF + 6-1 + 6-2
        "use_turn_separator": False,
        "turn_separator": " [SEP] ",
        "tfidf_top_k": 5,
    },
}

def generate_phase6_keyword_submission(model, tokenizer, test_df, config, output_path):
    """Phase 6 keyword_aware 스타일 (TF-IDF + 6-1 + 6-2)"""
    
    device = next(model.parameters()).device
    
    regex_pattern_config = config.get('regex_pattern', {
        "use_speaker_count": True,
        "use_pii_instruction": True,
        "show_pii_examples": True,
    })
    
    # TF-IDF extractor 포함
    preprocessor = Preprocess(
        loaded_config['tokenizer']['bos_token'],
        loaded_config['tokenizer']['eos_token'],
        prompt_config=config['prompt'],
        preprocess_config=config['preprocess'],
        tfidf_extractor=tfidf_extractor,  # TF-IDF 사용
        regex_pattern_config=regex_pattern_config,
    )
    
    gen_config = config['generation']
    summaries = []
    
    for idx in tqdm(range(len(test_df)), desc="추론 (keyword_aware)"):
        dialogue = test_df.iloc[idx]['dialogue']
        processed_dialogue = preprocessor.full_preprocess_pipeline(dialogue)
        
        inputs = tokenizer(
            processed_dialogue,
            max_length=loaded_config['tokenizer']['encoder_max_len'],
            truncation=True,
            return_tensors='pt'
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                num_beams=gen_config['num_beams'],
                length_penalty=gen_config['length_penalty'],
                repetition_penalty=gen_config['repetition_penalty'],
                no_repeat_ngram_size=gen_config['no_repeat_ngram_size'],
                max_length=gen_config['max_length'],
                early_stopping=gen_config['early_stopping'],
            )
        
        summary = decode_with_speaker_tags(tokenizer, generated_ids[0])
        summary = postprocess_summary(summary)
        summaries.append(summary)
    
    submission = pd.DataFrame({
        'fname': test_df['fname'],
        'summary': summaries
    })
    
    submission.to_csv(output_path, index=False)
    print(f"\n✅ 저장 완료: {output_path}")
    print(f"   평균 길이: {np.mean([len(s) for s in summaries]):.1f}자")
    
    has_speaker_tags = sum(1 for s in summaries if '#Person' in s)
    print(f"   화자 태그 포함: {has_speaker_tags}/{len(summaries)}개 ({has_speaker_tags/len(summaries)*100:.1f}%)")
    
    return submission

print("\n🚀 Phase 6 keyword_aware 제출 파일 생성 중...")
submission_phase6_keyword = generate_phase6_keyword_submission(
    eval_model, eval_tokenizer, test_df_phase6_submit,
    PHASE6_KEYWORD_CONFIG,
    "./prediction/submit_phase6_keyword.csv"
)

# =============================================================================
# 샘플 비교 출력
# =============================================================================

print("\n" + "=" * 60)
print("📋 Phase 6 제출 샘플 비교:")
print("=" * 60)

# PII 마스킹이 있는 Test 샘플 찾기
test_with_pii = []
for i, dialogue in enumerate(test_df_phase6_submit['dialogue']):
    if get_pii_mask_tokens(dialogue):
        test_with_pii.append(i)
        if len(test_with_pii) >= 3:
            break

print(f"\nPII 마스킹이 있는 Test 샘플: {test_with_pii}")

for i in test_with_pii[:2]:
    dialogue = test_df_phase6_submit.iloc[i]['dialogue']
    pii_masks = get_pii_mask_tokens(dialogue)
    
    print(f"\n[Test {i}] PII 마스킹: {pii_masks}")
    print(f"  Balanced: {submission_phase6['summary'].iloc[i][:100]}...")
    print(f"  Keyword:  {submission_phase6_keyword['summary'].iloc[i][:100]}...")

# 기존 제출과 비교
print("\n📋 기존 최적화 제출과 비교:")
try:
    old_submission = pd.read_csv("./prediction/submit_optimized_v2.csv")
    for i in range(3):
        print(f"\n[Test {i}]")
        print(f"  기존:     {old_submission['summary'].iloc[i][:70]}...")
        print(f"  Phase6:   {submission_phase6['summary'].iloc[i][:70]}...")
except:
    print("   기존 제출 파일을 찾을 수 없습니다.")

print("\n" + "=" * 80)
print("✅ Phase 6 제출 파일 생성 완료!")
print("=" * 80)
print("""
📤 생성된 제출 파일:
  1. ./prediction/submit_phase6_balanced.csv
     - 6-1 화자 수 명시 + 6-2 PII 마스킹 유지 지시
     
  2. ./prediction/submit_phase6_keyword.csv
     - TF-IDF 키워드 + 6-1 + 6-2 통합

🎯 기대 효과:
  - 6-1: "한 사람만 요약하는" 버그 감소
  - 6-2: PII 마스킹 토큰 보존으로 ROUGE 점수 향상

📝 리더보드 제출 순서:
  1) submit_phase6_balanced.csv → 기본 6-1, 6-2 전략
  2) submit_phase6_keyword.csv → TF-IDF 키워드 추가 효과 확인
""")